<a href="https://colab.research.google.com/github/kingmohamedtr-max/Mushroom-project-/blob/main/Mushroom_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business case (Mushroom dataset) - Supervised Machine learning

Target: (class: poisonous or not)

Business case: Help the FDA (food and drug admistration) ensure safety through toxicity detection for food and drug products that contain mushrooms.

Edible = e, poisonous = p
Audience: FDA

By: Mohamed Traore, Francesca Saba, Paulina Mauran and Katelyn Olifiant

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Import pandas
import pandas as pd

# 3. Load your CSV
# Make sure you update the path to the correct folder where your file is stored
file_path = "/content/drive/My Drive/secondary_data mushrooms.csv"

df = pd.read_csv(file_path)

# 4. Display the dataframe
df.head()

# EDA

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
# Check for missing values
df.isnull().sum()


In [ ]:
missing_values = df.isnull().sum()
sorted_missing_values = missing_values.sort_values(ascending=False)
top_5_missing_columns = sorted_missing_values.head(5).index.tolist()

print("Top 5 columns with the most missing values:")
print(top_5_missing_columns)

# Drop the identified columns from the DataFrame
df = df.drop(columns=top_5_missing_columns)

print("\nDataFrame shape after dropping columns:", df.shape)

# Verify missing values again
df.isnull().sum()

In [ ]:
for column in df.columns:
    if df[column].isnull().any():
        mode_value = df[column].mode()[0] # Get the first mode if there are multiple
        df[column].fillna(mode_value, inplace=True)

print("Missing values after imputation:")
df.isnull().sum()

Data Analysis Key Findings
The top 5 columns with the most missing values were identified as ['veil-type', 'spore-print-color', 'veil-color', 'stem-root', 'stem-surface'].
These 5 columns were dropped from the DataFrame, reducing its shape from an implied (61069, 21) to (61069, 16).
After dropping the top 5 columns, missing values were still present in cap-surface, gill-attachment, gill-spacing, and ring-type.
All remaining missing values in the DataFrame were successfully imputed using the mode of their respective columns.
Verification confirmed that no missing values are left in the DataFrame after the imputation process.
Insights or Next Steps
The DataFrame is now cleaned of missing values, making it ready for further exploratory data analysis or machine learning model training.
The high number of missing values in the dropped columns (veil-type, spore-print-color, veil-color, stem-root, stem-surface) suggests these features might be unreliable or not universally applicable, which could be investigated further if their original data source allows.

Significant Missing Values Leading to Feature Removal: Five columns (veil-type, spore-print-color, veil-color, stem-root, stem-surface) had an extremely high percentage of missing values (some over 80%). These columns were subsequently dropped from the DataFrame. This implies that these features are either largely unrecorded or unreliable in the dataset. For modeling, this means these features cannot be used directly, simplifying the model but also potentially losing information if these features are truly important and could have been imputed with more sophisticated methods or external data. The remaining missing values were imputed using the mode, which is a simple strategy but might not be optimal for all features.

Positive Skewness and Outliers in Numerical Features: All three numerical features (cap-diameter, stem-height, and stem-width) exhibit positive skewness with numerous outliers on the higher end. Many machine learning algorithms assume normally distributed data or are sensitive to the presence of skewness and outliers (e.g., linear models, distance-based algorithms). Therefore, preprocessing steps such as log transformations or robust scaling (e.g., using RobustScaler or QuantileTransformer) will likely be crucial to mitigate the impact of skewness and outliers, potentially improving model performance and stability.

Univariate analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

print(f"Categorical columns identified: {list(categorical_cols)}")

# Plot count plots for categorical columns
for col in categorical_cols:
    print(f"\nAnalyzing categorical column: {col}")

    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index, palette='viridis')
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

print(f"Numerical columns identified: {list(numerical_cols)}")

# Plot histograms and boxplots for numerical columns
for col in numerical_cols:
    print(f"\nAnalyzing numerical column: {col}")

    plt.figure(figsize=(14, 6))

    # Histogram
    plt.subplot(1, 2, 1) # 1 row, 2 columns, 1st plot
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

    # Boxplot
    plt.subplot(1, 2, 2) # 1 row, 2 columns, 2nd plot
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot of {col}')
    plt.ylabel(col)

    plt.tight_layout()
    plt.show()

Skewness Analysis of Numerical Variables
Skewness refers to the asymmetry in a data distribution. A distribution is positively skewed (or right-skewed) if the tail is longer on the right side, meaning there are more extreme values on the higher end. A distribution is negatively skewed (or left-skewed) if the tail is longer on the left side, indicating more extreme values on the lower end.

Here's the breakdown for your numerical variables:

cap-diameter: The histogram for cap-diameter shows a long tail extending to the right, and the boxplot indicates numerous outliers on the upper end. This suggests that cap-diameter is positively skewed. Most mushrooms have smaller cap diameters, but there are a considerable number with much larger ones.

stem-height: Similar to cap-diameter, the stem-height histogram exhibits a tail to the right, and its boxplot shows outliers on the higher side. Therefore, stem-height is also positively skewed. Most mushrooms have shorter stems, but some have notably taller stems.

stem-width: The histogram for stem-width also displays a distinct rightward tail, and the boxplot points to many outliers with greater widths. This indicates that stem-width is positively skewed. The majority of mushrooms have narrower stems, with a smaller proportion having much thicker stems.

In summary, all three numerical variables (cap-diameter, stem-height, and stem-width) are positively skewed, meaning they have a concentration of data points at the lower end of their ranges and a few extreme values at the higher end. This is common in many natural measurements where a baseline exists, but occasional larger instances occur.

# Bivariate analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'class' is the target variable
target_variable = 'class'

# Identify categorical columns, excluding the target variable itself
categorical_cols = df.select_dtypes(include=['object']).columns.drop(target_variable, errors='ignore')

print(f"Analyzing categorical columns against target '{target_variable}': {list(categorical_cols)}")

# Plot count plots for each categorical column against the target
for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=col, hue=target_variable, palette='viridis', order=df[col].value_counts().index)
    plt.title(f'Distribution of {col} by {target_variable}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Note: The request also mentioned 'Stacked bar plots or interaction plots'.
# The countplots with 'hue' are a form of grouped bar plots showing interaction.
# If a more complex interaction plot (e.g., between two independent categorical features vs. target) is desired,
# please specify the features.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=col, order=df[col].value_counts().index, palette='viridis')
    plt.title(f'Bar Plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

sns.pairplot(df[numerical_cols])
plt.suptitle('Pair Plot of Numerical Features', y=1.02) # Adjust suptitle to not overlap
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify numerical columns (assuming 'df' is already loaded)
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

print(f"Numerical columns identified: {list(numerical_cols)}")

# Plot boxplots for numerical columns
for col in numerical_cols:
    print(f"\nAnalyzing numerical column: {col}")

    plt.figure(figsize=(7, 5))
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot of {col}')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# Multivariate analysis

In [ ]:
numerical_df = df.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numerical_df.corr()
display(correlation_matrix)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

Documentation of the correlation matrix

cap-diameter and stem-width: These two features exhibit the strongest positive correlation, with a coefficient of approximately 0.70. This suggests a robust relationship where mushrooms with larger cap diameters tend to have wider stems.

stem-height and stem-width: Following closely, there's a positive correlation of about 0.58 between stem height and stem width. This indicates that taller mushroom stems are generally associated with wider stems.

cap-diameter and stem-height: The weakest, though still positive, correlation is observed between cap diameter and stem height, with a coefficient of approximately 0.50. This means that while larger cap diameters are generally linked to taller stems, this relationship is less pronounced compared to the other pairs.

In summary, the dimensions of the mushroom (cap diameter, stem height, and stem width) all tend to increase together, but the relationship is strongest between cap-diameter and stem-width.

# Outliers

In [ ]:
import pandas as pd

print("Shape before outlier removal:", df.shape)

# Identify numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Iteratively remove outliers until none remain
max_iterations = 10
iteration = 0

while iteration < max_iterations:
    initial_shape = df.shape[0]
    mask = pd.Series([True] * len(df), index=df.index)

    for column in numerical_cols:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        column_mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)
        mask = mask & column_mask

    df = df[mask].copy()
    df.reset_index(drop=True, inplace=True)

    rows_removed = initial_shape - df.shape[0]
    print(f"Iteration {iteration + 1}: Removed {rows_removed} rows")

    if rows_removed == 0:
        print("No more outliers detected!")
        break

    iteration += 1

print("\nFinal shape after outlier removal:", df.shape)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Identify numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

print("=== Final Outlier Verification ===\n")

total_outliers = 0

for column in numerical_cols:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Count outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outlier_count = len(outliers)
    total_outliers += outlier_count

    status = "✓ CLEAN" if outlier_count == 0 else "✗ OUTLIERS FOUND"
    print(f"{column}: {outlier_count} outliers {status}")

    # Visualize
    plt.figure(figsize=(7, 5))
    sns.boxplot(y=df[column])
    plt.title(f'{column} - {status}')
    plt.ylabel(column)
    plt.tight_layout()
    plt.show()

print(f"\n{'='*50}")
if total_outliers == 0:
    print("✓ SUCCESS: All outliers removed! Boxplots are clean.")
else:
    print(f"✗ WARNING: {total_outliers} total outliers still present.")
print(f"{'='*50}")


In [ ]:
# Check current state
print("Current df shape:", df.shape)
print("\nColumn names in df:")
print(list(df.columns))

# Verify if columns were actually dropped
expected_dropped = ['veil-type', 'spore-print-color', 'veil-color', 'stem-root', 'stem-surface']

print("\nChecking if columns were dropped:")
for col in expected_dropped:
    if col in df.columns:
        print(f"  ✗ {col} - STILL PRESENT (not dropped)")
    else:
        print(f"  ✓ {col} - Dropped successfully")

print("\nMissing values per column:")
print(df.isnull().sum().sort_values(ascending=False))


# logisitc regression

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# 1. Separate features (X) from the target variable (y)
X = df.drop('class', axis=1)
y = df['class']

# 2. Identify numerical columns
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(f"Numerical features: {numerical_features}")

# 3. Identify categorical columns
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical features: {categorical_features}")

# 4. and 5. Create a ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 6. Apply the preprocessor to X
X_processed = preprocessor.fit_transform(X)

print("\nShape of original X:", X.shape)
print("Shape of processed X (X_processed):", X_processed.shape)
print("Target variable y head:\n", y.head())

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting sets to verify the split
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate a Logistic Regression model
log_reg_model = LogisticRegression(random_state=42, max_iter=1000)

# Train the model using the training data
log_reg_model.fit(X_train, y_train)

print("Logistic Regression model trained successfully.")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Make predictions on the test set
y_pred = log_reg_model.predict(X_test)

# Determine the index of the positive class ('p') in the model's classes
positive_class_idx = list(log_reg_model.classes_).index('p')
y_pred_proba = log_reg_model.predict_proba(X_test)[:, positive_class_idx] # Probability of the positive class (class 'p')

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='p')
recall = recall_score(y_test, y_pred, pos_label='p')
f1 = f1_score(y_test, y_pred, pos_label='p')

# For roc_auc_score, y_true must be binary (0 or 1). Convert 'p' to 1 and 'e' to 0.
# Ensure y_test is mapped correctly to 0 and 1, where 1 is the positive class 'p'
y_test_binary = (y_test == 'p').astype(int)
roc_auc = roc_auc_score(y_test_binary, y_pred_proba)

conf_matrix = confusion_matrix(y_test, y_pred, labels=['e', 'p'])
class_report = classification_report(y_test, y_pred)

print(f"\nModel Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"\nConfusion Matrix:\n{conf_matrix}")
print(f"\nClassification Report:\n{class_report}")

## Logistic Regression Model Performance Summary

The Logistic Regression model has been evaluated on the test set, and its performance metrics provide a good understanding of its ability to classify mushrooms as poisonous ('p') or edible ('e').

### 1. Overall Performance
The model achieved an **Accuracy of 0.8069**, meaning it correctly classified about 80.69% of the mushrooms in the test set. However, accuracy alone can be misleading, especially with imbalanced datasets or when certain types of errors are more critical than others, as is the case here.

### 2. Metric Breakdown and Business Case Relevance

*   **Precision (for poisonous 'p'): 0.8189**
    *   This means that when the model predicts a mushroom is poisonous, it is correct 81.89% of the time. In the context of the FDA ensuring safety, high precision for 'p' is important because it reduces the number of false alarms (labeling an edible mushroom as poisonous). While false alarms can lead to unnecessary product recalls or waste, they are generally less critical than missing a poisonous product.

*   **Recall (for poisonous 'p'): 0.8287**
    *   This indicates that the model correctly identified 82.87% of all actual poisonous mushrooms. For the FDA's safety mission, **recall for 'p' is paramount**. A high recall minimizes the number of false negatives – actual poisonous mushrooms that the model incorrectly classifies as edible. Missing a poisonous mushroom could have severe public health consequences.

*   **F1-Score (for poisonous 'p'): 0.8238**
    *   The F1-Score is the harmonic mean of precision and recall, providing a balance between the two. A score of 0.8238 suggests a reasonably good balance, but considering the high stakes of false negatives, recall might be a more critical single metric.

*   **ROC-AUC: 0.8851**
    *   The Area Under the Receiver Operating Characteristic Curve (ROC-AUC) measures the model's ability to distinguish between classes. An ROC-AUC of 0.8851 indicates a good discriminative power, suggesting the model is generally good at ranking poisonous mushrooms higher than edible ones.

### 3. Key Insights from the Confusion Matrix
The confusion matrix is a critical tool for understanding where the model makes mistakes:

```
[[3600 1011]
 [ 945 4573]]
```
(Rows represent actual classes: 'e' then 'p'. Columns represent predicted classes: 'e' then 'p'.)

*   **True Negatives (TN): 3600**
    *   The model correctly identified 3600 edible mushrooms as edible ('e'). These are correctly classified safe products.

*   **False Positives (FP) / Type I Error: 1011**
    *   The model incorrectly classified 1011 edible mushrooms as poisonous ('p'). These are false alarms. While not ideal for business (e.g., product waste), they don't pose a direct health risk.

*   **False Negatives (FN) / Type II Error: 945**
    *   The model incorrectly classified 945 poisonous mushrooms as edible ('e'). **This is the most critical error type for the FDA's business case.** These are potentially harmful products that the model deemed safe, posing a direct threat to public health.

*   **True Positives (TP): 4573**
    *   The model correctly identified 4573 poisonous mushrooms as poisonous ('p'). These are successfully detected dangerous products.

### 4. Suitability for the Business Case
Given the business case of the FDA ensuring safety, the model's performance, particularly the **945 False Negatives**, is a significant concern. While an F1-score of 0.82 and ROC-AUC of 0.88 are generally good, the high cost of a Type II error (missing a poisonous mushroom) means that even this number of false negatives is unacceptable for a safety-critical application. If 945 poisonous mushrooms are wrongly labeled as edible in the real world, it could lead to serious health issues.

**Conclusion:** The current Logistic Regression model, despite its decent overall metrics, is **not yet suitable** for direct deployment in a safety-critical environment like FDA toxicity detection due to the high number of false negatives. The priority for this business case should be to **minimize False Negatives** even if it means accepting a higher rate of False Positives. Further model tuning, exploring different algorithms, or collecting more relevant features might be necessary to improve the recall for the poisonous class significantly.

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=log_reg_model.classes_, yticklabels=log_reg_model.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Feature selction

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# scale the dataset
Scale = MinMaxScaler

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1. Instantiate a LabelEncoder object
label_encoder = LabelEncoder()

# 2. Fit the LabelEncoder to the target variable y and then transform y
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Import necessary modules for feature selection
from sklearn.feature_selection import SelectKBest, f_regression

# Use the cleaned 'df' available in the kernel state
# Separate features (X_kbest) and target (y_kbest) from the cleaned df
X_kbest = df.drop('class', axis=1)
y_kbest = df['class']

# Encode the target variable if not already encoded for this specific run
# Using the existing label_encoder from cell acab400e
y_kbest_encoded = label_encoder.transform(y_kbest)

# Apply the preprocessor (from cell acab400e) to X_kbest to get numerical features
X_kbest_processed = preprocessor.fit_transform(X_kbest)

# Apply k-best
k = 30
selector = SelectKBest(score_func=f_regression, k=k) # f_regression expects numerical input
X_new = selector.fit_transform(X_kbest_processed, y_kbest_encoded)

# feature selection
selected_indices = selector.get_support(indices=True)

# Get feature names after preprocessing
# The 'preprocessor' is a ColumnTransformer defined in cell acab400e
processed_feature_names = []
for name, transformer, cols in preprocessor.transformers_:
    if name == 'num':
        processed_feature_names.extend(cols)
    elif name == 'cat': # Assuming 'cat' is the name of your OneHotEncoder transformer
        processed_feature_names.extend(transformer.get_feature_names_out(cols))

all_feature_names = processed_feature_names # This is the list of all features after preprocessing

selected_feature_names = [all_feature_names[i] for i in selected_indices]

# print features
print("Selected Features:", selected_feature_names)

## K-Best Feature Selection (`k=30`)

Using `SelectKBest` with the `f_regression` scoring function to select the top 30 features from the preprocessed dataset, the following features were identified as most influential:

*   `cap-diameter`
*   `stem-height`
*   `stem-width`
*   `cap-shape_b`
*   `cap-shape_o`
*   `cap-surface_i`
*   `cap-surface_k`
*   `cap-surface_s`
*   `cap-color_b`
*   `cap-color_e`
*   `cap-color_n`
*   `cap-color_r`
*   `gill-attachment_a`
*   `gill-attachment_e`
*   `gill-attachment_p`
*   `gill-spacing_c`
*   `gill-spacing_d`
*   `gill-color_b`
*   `gill-color_n`
*   `gill-color_w`
*   `stem-color_k`
*   `stem-color_n`
*   `stem-color_p`
*   `stem-color_w`
*   `stem-color_y`
*   `ring-type_g`
*   `ring-type_z`
*   `habitat_g`
*   `habitat_w`
*   `season_w`

This selection provides a broader set of features that contribute significantly to predicting the target variable.

- k = 15

In [ ]:
# Import necessary modules for feature selection
from sklearn.feature_selection import SelectKBest, f_regression

# Use the cleaned 'df' available in the kernel state
# Separate features (X_kbest) and target (y_kbest) from the cleaned df
X_kbest = df.drop('class', axis=1)
y_kbest = df['class']

# Encode the target variable if not already encoded for this specific run
# Using the existing label_encoder from cell acab400e
y_kbest_encoded = label_encoder.transform(y_kbest)

# Apply the preprocessor (from cell acab400e) to X_kbest to get numerical features
X_kbest_processed = preprocessor.fit_transform(X_kbest)

# Apply k-best
k = 15
selector = SelectKBest(score_func=f_regression, k=k) # f_regression expects numerical input
X_new = selector.fit_transform(X_kbest_processed, y_kbest_encoded)

# feature selection
selected_indices = selector.get_support(indices=True)

# Get feature names after preprocessing
# The 'preprocessor' is a ColumnTransformer defined in cell acab400e
processed_feature_names = []
for name, transformer, cols in preprocessor.transformers_:
    if name == 'num':
        processed_feature_names.extend(cols)
    elif name == 'cat': # Assuming 'cat' is the name of your OneHotEncoder transformer
        processed_feature_names.extend(transformer.get_feature_names_out(cols))

all_feature_names = processed_feature_names # This is the list of all features after preprocessing

selected_feature_names = [all_feature_names[i] for i in selected_indices]

# print features
print("Selected Features:", selected_feature_names)

K-Best Feature Selection (k=15)
K-Best feature selection is a statistical method that selects the top k features based on a scoring function. In this case, f_regression was used as the scoring function, which computes F-values for regression tasks. It evaluates the linear dependency between each feature and the target variable.

For k=15, the following features were selected from the preprocessed dataset:

cap-diameter
stem-width
cap-shape_b
cap-surface_k
cap-color_e
gill-attachment_a
gill-attachment_p
gill-spacing_c
gill-spacing_d
gill-color_n
gill-color_w
stem-color_w
ring-type_z
habitat_g
season_w

- k = 8

In [ ]:
# Import necessary modules for feature selection
from sklearn.feature_selection import SelectKBest, f_regression

# Use the cleaned 'df' available in the kernel state
# Separate features (X_kbest) and target (y_kbest) from the cleaned df
X_kbest = df.drop('class', axis=1)
y_kbest = df['class']

# Encode the target variable if not already encoded for this specific run
# Using the existing label_encoder from cell acab400e
y_kbest_encoded = label_encoder.transform(y_kbest)

# Apply the preprocessor (from cell acab400e) to X_kbest to get numerical features
X_kbest_processed = preprocessor.fit_transform(X_kbest)

# Apply k-best
k = 8
selector = SelectKBest(score_func=f_regression, k=k) # f_regression expects numerical input
X_new = selector.fit_transform(X_kbest_processed, y_kbest_encoded)

# feature selection
selected_indices = selector.get_support(indices=True)

# Get feature names after preprocessing
# The 'preprocessor' is a ColumnTransformer defined in cell acab400e
processed_feature_names = []
for name, transformer, cols in preprocessor.transformers_:
    if name == 'num':
        processed_feature_names.extend(cols)
    elif name == 'cat': # Assuming 'cat' is the name of your OneHotEncoder transformer
        processed_feature_names.extend(transformer.get_feature_names_out(cols))

all_feature_names = processed_feature_names # This is the list of all features after preprocessing

selected_feature_names = [all_feature_names[i] for i in selected_indices]

# print features
print("Selected Features:", selected_feature_names)

K-Best Feature Selection (k=8)
K-Best feature selection is a technique that selects the top k features based on a statistical test. It's part of the sklearn.feature_selection module and helps in reducing dimensionality by picking the features with the strongest relationship to the target variable. For this analysis, k was set to 8.

The f_regression scoring function was used as the statistical test. f_regression computes the F-value for each feature, which measures the linear dependency between the feature and the target variable. A higher F-value indicates a stronger relationship. This method is suitable for numerical features and a continuous target, or a binary target treated numerically (as is the case after Label Encoding the 'class' column).

When k=8, the following features were selected:

cap-diameter
stem-width
cap-surface_k
gill-attachment_p
gill-spacing_d
gill-color_w
stem-color_w
ring-type_z

- k = 4

In [ ]:
# Import necessary modules for feature selection
from sklearn.feature_selection import SelectKBest, f_regression

# Use the cleaned 'df' available in the kernel state
# Separate features (X_kbest) and target (y_kbest) from the cleaned df
X_kbest = df.drop('class', axis=1)
y_kbest = df['class']

# Encode the target variable if not already encoded for this specific run
# Using the existing label_encoder from cell acab400e
y_kbest_encoded = label_encoder.transform(y_kbest)

# Apply the preprocessor (from cell acab400e) to X_kbest to get numerical features
X_kbest_processed = preprocessor.fit_transform(X_kbest)

# Apply k-best
k = 4
selector = SelectKBest(score_func=f_regression, k=k) # f_regression expects numerical input
X_new = selector.fit_transform(X_kbest_processed, y_kbest_encoded)

# feature selection
selected_indices = selector.get_support(indices=True)

# Get feature names after preprocessing
# The 'preprocessor' is a ColumnTransformer defined in cell acab400e
processed_feature_names = []
for name, transformer, cols in preprocessor.transformers_:
    if name == 'num':
        processed_feature_names.extend(cols)
    elif name == 'cat': # Assuming 'cat' is the name of your OneHotEncoder transformer
        processed_feature_names.extend(transformer.get_feature_names_out(cols))

all_feature_names = processed_feature_names # This is the list of all features after preprocessing

selected_feature_names = [all_feature_names[i] for i in selected_indices]

# print features
print("Selected Features:", selected_feature_names)

K-Best Feature Selection (k=4) Summary
Process:
The K-Best feature selection method, in conjunction with f_regression as the scoring function, was employed to identify the most relevant features for predicting the target variable. f_regression computes the F-value for each feature, which measures the linear dependency between the feature and the target. A higher F-value indicates a stronger relationship.

For this specific selection, k was set to 4, meaning the top 4 features with the highest F-values were chosen.

Selected Features (k=4):
Based on the f_regression scoring and selecting the top 4 features, the following features were identified:

cap-diameter
stem-width
gill-attachment_p
stem-color_w



Data Analysis Key Findings
For k=30, 30 features were selected, including physical dimensions (cap-diameter, stem-height, stem-width) and numerous one-hot encoded categorical features related to cap shape, surface, color, gill attachment, spacing, color, stem color, ring type, habitat, and season.
For k=15, the selection narrowed to 15 features. This set still included cap-diameter and stem-width, along with specific categorical indicators like cap-shape_b, cap-surface_k, gill-attachment_a, gill-attachment_p, gill-spacing_c, gill-spacing_d, gill-color_n, gill-color_w, stem-color_w, ring-type_z, habitat_g, and season_w.
For k=8, only 8 features were chosen: cap-diameter, stem-width, cap-surface_k, gill-attachment_p, gill-spacing_d, gill-color_w, stem-color_w, and ring-type_z.
For k=4, the most restrictive selection yielded the top 4 features: cap-diameter, stem-width, gill-attachment_p, and stem-color_w. These four features consistently appeared in all selections regardless of the k value.
Insights or Next Steps
The features cap-diameter, stem-width, gill-attachment_p, and stem-color_w are highly important for predicting the target variable, given their consistent selection across all k values.
Further model training and evaluation should be performed using these different sets of selected features to determine the optimal k value that balances model complexity and predictive performance.


# Lasso

- aplha = 0.08

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np

X_lasso_features = X_kbest_processed
y_lasso_target = y_kbest_encoded

# Scale the features (Lasso is sensitive to feature scaling)
# Pass with_mean=False as X_lasso_features might be a sparse matrix
scaler = StandardScaler(with_mean=False)
X_lasso_scaled = scaler.fit_transform(X_lasso_features)

# Define the alpha value (you can change this to experiment)
user_alpha = 0.08# Example alpha value, you can change this

# Apply Lasso for feature selection
lasso_model = Lasso(alpha=user_alpha, random_state=42, max_iter=10000)
lasso_model.fit(X_lasso_scaled, y_lasso_target)

print(f"Lasso model applied with alpha: {user_alpha}")

# Identify selected features (non-zero coefficients)
selected_lasso_indices = np.where(lasso_model.coef_ != 0)[0]

# Get feature names after preprocessing (using all_feature_names from previous steps)
selected_lasso_features = [all_feature_names[i] for i in selected_lasso_indices]

print("\nSelected Features by Lasso:")
print(selected_lasso_features)

alpha = 0.05

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np

X_lasso_features = X_kbest_processed
y_lasso_target = y_kbest_encoded

# Scale the features (Lasso is sensitive to feature scaling)
# Pass with_mean=False as X_lasso_features might be a sparse matrix
scaler = StandardScaler(with_mean=False)
X_lasso_scaled = scaler.fit_transform(X_lasso_features)

# Define the alpha value (you can change this to experiment)
user_alpha = 0.05 # Example alpha value, you can change this

# Apply Lasso for feature selection
lasso_model = Lasso(alpha=user_alpha, random_state=42, max_iter=10000)
lasso_model.fit(X_lasso_scaled, y_lasso_target)

print(f"Lasso model applied with alpha: {user_alpha}")

# Identify selected features (non-zero coefficients)
selected_lasso_indices = np.where(lasso_model.coef_ != 0)[0]

# Get feature names after preprocessing (using all_feature_names from previous steps)
selected_lasso_features = [all_feature_names[i] for i in selected_lasso_indices]

print("\nSelected Features by Lasso:")
print(selected_lasso_features)

alpha = 0.03

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np

X_lasso_features = X_kbest_processed
y_lasso_target = y_kbest_encoded

# Scale the features (Lasso is sensitive to feature scaling)
# Pass with_mean=False as X_lasso_features might be a sparse matrix
scaler = StandardScaler(with_mean=False)
X_lasso_scaled = scaler.fit_transform(X_lasso_features)

# Define the alpha value (you can change this to experiment)
user_alpha = 0.03 # Example alpha value, you can change this

# Apply Lasso for feature selection
lasso_model = Lasso(alpha=user_alpha, random_state=42, max_iter=10000)
lasso_model.fit(X_lasso_scaled, y_lasso_target)

print(f"Lasso model applied with alpha: {user_alpha}")

# Identify selected features (non-zero coefficients)
selected_lasso_indices = np.where(lasso_model.coef_ != 0)[0]

# Get feature names after preprocessing (using all_feature_names from previous steps)
selected_lasso_features = [all_feature_names[i] for i in selected_lasso_indices]

print("\nSelected Features by Lasso:")
print(selected_lasso_features)

## Document Lasso (alpha=0.08)



### Lasso Feature Selection Process

Lasso (Least Absolute Shrinkage and Selection Operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the statistical model it produces. It adds a penalty equivalent to the absolute value of the magnitude of coefficients to the loss function. This type of regularization (L1 regularization) has the effect of shrinking some coefficients exactly to zero. When a coefficient becomes zero, the corresponding feature is effectively removed from the model, thus performing feature selection.

### Features Selected with `alpha = 0.08`

With a regularization strength (`alpha`) set to **0.08**, the Lasso model identified the following features as most relevant (i.e., their coefficients were not shrunk to zero):

*   `stem-width`
*   `gill-attachment_p`
*   `stem-color_w`
*   `ring-type_z`

## Document Lasso (alpha=0.05)

### Lasso Feature Selection Process
Lasso (Least Absolute Shrinkage and Selection Operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the statistical model it produces. It achieves this by adding a penalty equivalent to the absolute value of the magnitude of coefficients to the loss function. This L1 regularization technique can shrink some coefficients exactly to zero, effectively performing automatic feature selection by excluding less important features from the model.

### Features Selected with `alpha = 0.05`
With a regularization strength (`alpha`) set to **0.05**, the Lasso model identified the following features as most relevant (i.e., their coefficients were not shrunk to zero):

*   cap-diameter
*   stem-width
*   cap-surface_k
*   cap-color_e
*   cap-color_n
*   cap-color_r
*   gill-attachment_p
*   gill-spacing_c
*   gill-spacing_d
*   gill-color_w
*   stem-color_w
*   ring-type_z
*   season_w

## Document Lasso (alpha=0.03)

### Lasso Feature Selection Process
Lasso (Least Absolute Shrinkage and Selection Operator) is a regression analysis method that performs both variable selection and regularization to enhance the prediction accuracy and interpretability of the statistical model. It uses L1 regularization, which adds a penalty equal to the absolute value of the magnitude of coefficients to the loss function. This type of regularization has the effect of shrinking some coefficients to exactly zero, effectively performing feature selection by excluding those features from the model.

The `alpha` parameter controls the strength of this penalty. A higher `alpha` value means more regularization and more coefficients being driven to zero, leading to a sparser model (fewer selected features). Conversely, a lower `alpha` value means less regularization and more features being retained.

### Selected Features with alpha = 0.03
When the regularization strength (`alpha`) was set to 0.03, the Lasso model identified the following features as most relevant for prediction:

*   `cap-diameter`
*   `stem-width`
*   `cap-shape_b`
*   `cap-shape_o`
*   `cap-surface_h`
*   `cap-surface_i`
*   `cap-surface_k`
*   `cap-surface_s`
*   `cap-color_b`
*   `cap-color_e`
*   `cap-color_n`
*   `cap-color_r`
*   `gill-attachment_e`
*   `gill-attachment_p`
*   `gill-spacing_c`
*   `gill-spacing_d`
*   `gill-color_n`
*   `gill-color_w`
*   `stem-color_g`
*   `stem-color_p`
*   `stem-color_w`
*   `ring-type_g`
*   `ring-type_z`
*   `habitat_g`
*   `habitat_l`
*   `habitat_w`
*   `season_s`
*   `season_w`

### Data Analysis Key Findings
*   **Impact of Regularization Strength**: As the Lasso regularization strength (alpha) decreased from 0.08 to 0.03, the number of selected features progressively increased, indicating that weaker regularization retains more features. Specifically, `alpha = 0.08` selected 4 features, `alpha = 0.05` selected 13 features, and `alpha = 0.03` selected 28 features.
*   **Consistently Selected Features**: The features `stem-width`, `gill-attachment_p`, `stem-color_w`, and `ring-type_z` were consistently selected across all tested alpha values (0.08, 0.05, and 0.03), suggesting their strong relevance to the model.
*   **Increasing Feature Inclusion with Lower Alpha**: With `alpha = 0.05`, features like `cap-diameter`, `cap-surface_k`, `cap-color_e`, `cap-color_n`, `cap-color_r`, `gill-spacing_c`, `gill-spacing_d`, `gill-color_w`, and `season_w` were included in addition to the ones selected at `alpha = 0.08`. At `alpha = 0.03`, an even broader set of features including various `cap-shape`, `cap-surface`, `cap-color`, `gill-attachment`, `gill-spacing`, `gill-color`, `stem-color`, `ring-type`, `habitat`, and `season` categories were deemed relevant.

### Insights or Next Steps
*   The analysis clearly illustrates the trade-off between model simplicity (fewer features) and potential predictive power as regularization strength is adjusted. Selecting an optimal alpha value requires balancing these aspects, often through cross-validation.
*   The consistently selected features are strong candidates for inclusion in simpler, more interpretable models or for further domain-specific investigation, as they demonstrate robustness across varying regularization penalties.


# Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Changed to Classifier for classification task
import pandas as pd

# Reuse preprocessed data from previous steps
X_dtree = X_kbest_processed
y_dtree = y_kbest_encoded

# Initialize and fit the Decision Tree Classifier
tree_classifier = DecisionTreeClassifier(random_state=42) # Added random_state for reproducibility
tree_classifier.fit(X_dtree, y_dtree)

# Get feature importances
feature_importances = tree_classifier.feature_importances_ # Corrected typo

# Create a DataFrame for better visualization and sorting
importance_df = pd.DataFrame({
    'Feature': all_feature_names, # Using all_feature_names from preprocessing
    'Importance': feature_importances
})

# Sort features by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print sorted features and their importances
print("Decision Tree Feature Importances:")
for index, row in importance_df.iterrows():
    print(f"{row['Feature']}: {row['Importance']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate dynamic figure height: 0.35 inches per feature usually works well
fig_height = len(importance_df) * 0.35

plt.figure(figsize=(12, fig_height))

sns.barplot(x='Importance', y='Feature', hue='Feature', data=importance_df, palette='viridis', legend=False)
plt.title('Decision Tree Feature Importances (All Features)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# ----- 1. Create train/test split for the FULL dataset -----
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_kbest_processed,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ----- 2. Fit a shallow tree for visualization -----
viz_dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)
viz_dt_model.fit(X_train_full, y_train_full)

# ----- 3. Plot the tree -----
plt.figure(figsize=(25, 15))
plot_tree(
    viz_dt_model,
    filled=True,
    rounded=True,
    fontsize=10,
    feature_names=all_feature_names,
    class_names=label_encoder.classes_,
)

plt.title("Decision Tree Classifier Visualization (Max Depth = 3)")
plt.show()

Decision Tree Graph
The Decision Tree graph provides a visual representation of how the model makes classifications based on your features. Here's a breakdown of its components and how to interpret it:

Nodes: Each box in the tree is called a node.

Root Node: The very top node is where the decision-making process begins.
Internal (Decision) Nodes: These are nodes that have branches leading to other nodes. They contain a condition (e.g., stem-width <= X.XX) that splits the data. If the condition is true, the data goes down one path; if false, it goes down the other.
Leaf Nodes: These are the terminal nodes of the tree (they don't branch further). They contain the final prediction (class) for the samples that reach that node.
Split Conditions: Each internal node contains a condition based on a feature (e.g., stem-width <= 0.437). Samples are directed left or right based on whether they satisfy this condition.

gini: This value represents the Gini impurity of the node. Gini impurity is a measure of the purity of the node. A gini of 0 means the node is perfectly pure (all samples belong to the same class), while a higher gini (up to 0.5 for binary classification) indicates a more mixed set of classes. The tree aims to find splits that reduce Gini impurity.

samples: This indicates the number of samples that reached this particular node.

value: This is an array showing the count of samples per class in that node. For example, value = [X, Y] means there are X samples of class 'e' (edible) and Y samples of class 'p' (poisonous) in that node.

class: This is the predicted class for the majority of samples in that node. In a leaf node, this is the final prediction for any sample reaching it.

max_depth=3: You explicitly set max_depth=3 when creating viz_dt_model. This means the tree will have at most 3 levels of decisions (excluding the root node itself), keeping the visualization simple and preventing overfitting for this visual example.

Feature Names (feature_names) and Class Names (class_names): The feature_names=all_feature_names and class_names=label_encoder.classes_ arguments ensure that the nodes are labeled with meaningful feature names and class labels ('e' for edible, 'p' for poisonous), making the tree easy to understand.

How to Interpret a Prediction: To predict the class of a new mushroom, you start at the root node and follow the path down the tree by evaluating the conditions at each internal node until you reach a leaf node. The class indicated in the leaf node is the model's prediction.



Based on the Decision Tree feature importances previously calculated and visualized, the features that exhibit the highest importance scores are considered the most influential in the model's decision-making process for classifying mushrooms as poisonous or edible.

Looking at the importance_df from the Decision Tree:

stem-width (0.1364)
stem-height (0.0795)
stem-color_w (0.0612)
gill-spacing_c (0.0588)
cap-diameter (0.0545)
These features are important because Decision Trees work by finding the best splits in the data to separate classes, and features with higher importance scores are those that lead to significant reductions in impurity (like Gini impurity) across the tree's nodes. Essentially, these features are frequently used early in the tree-building process to make crucial distinctions between mushroom classes. For example, stem-width and cap-diameter are numerical features that allow the tree to create thresholds that effectively divide data points, while stem-color_w and gill-spacing_c (likely representing specific categories of stem color and gill spacing) are categorical features that, when present or absent, strongly correlate with one class over another

# Chi-square

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
import pandas as pd
import numpy as np

# Get the names of the one-hot encoded categorical features from the preprocessor
# categorical_features and preprocessor are available from earlier steps
cat_transformer = preprocessor.named_transformers_['cat']
one_hot_categorical_feature_names = cat_transformer.get_feature_names_out(categorical_features)

# Create a mask for selecting only the one-hot encoded features from X_processed
# Assuming numerical features are at the beginning of X_processed
num_features_count = len(numerical_features)
# The one-hot encoded features start after the numerical features
X_categorical_processed = X_processed[:, num_features_count:]

# Ensure X_categorical_processed is non-negative for chi2 (which it should be after OneHotEncoding)
# and convert to dense array if it's a sparse matrix
if hasattr(X_categorical_processed, 'toarray'):
    X_categorical_processed_dense = X_categorical_processed.toarray()
else:
    X_categorical_processed_dense = X_categorical_processed

# Apply SelectKBest with chi2 scoring function
# We don't specify k here, just get scores for all categorical features
chi2_selector = SelectKBest(chi2, k='all')
chi2_selector.fit(X_categorical_processed_dense, y_encoded)

# Get the scores (chi2 statistics) and p-values
chi2_scores = chi2_selector.scores_
chi2_pvalues = chi2_selector.pvalues_

# Create a DataFrame to display the chi-square results
chi2_results_df = pd.DataFrame({
    'Feature': one_hot_categorical_feature_names,
    'Chi2_Score': chi2_scores,
    'P_Value': chi2_pvalues
})

# Sort by Chi2 Score in descending order
chi2_results_df = chi2_results_df.sort_values(by='Chi2_Score', ascending=False).reset_index(drop=True)

print("Chi-Square Test Results for Categorical Features (Sorted by Score):\n")
display(chi2_results_df)

## Chi-Square Feature Selection Documentation

### Method Overview
Chi-Square ($\\chi^2$) feature selection is a statistical test used in feature selection to determine the dependency between categorical variables. In the context of classification, it helps assess how strongly a categorical feature is related to the categorical target variable. The null hypothesis for the Chi-Square test states that there is no relationship between the two categorical variables (i.e., they are independent).

**How it works:**
- It calculates the Chi-Square statistic for each categorical feature against the target variable.
- A higher Chi-Square score indicates a stronger dependency between the feature and the target, suggesting that changes in the feature's categories are significantly associated with changes in the target variable's categories.
- A lower P-value indicates that the observed relationship is statistically significant, meaning we can reject the null hypothesis of independence.

### Application in this Analysis
In this analysis, `SelectKBest` with the `chi2` scoring function was applied to the one-hot encoded categorical features (`X_categorical_processed_dense`) against the numerically encoded target variable (`y_encoded`). The numerical features were excluded from this test as Chi-Square is specifically designed for categorical data.

### Key Findings from Chi-Square Test Results
From the `chi2_results_df`, sorted by `Chi2_Score`:

*   **Highest Importance**: Features such as `gill-attachment_p`, `ring-type_z`, and `stem-color_w` exhibit the highest Chi2 scores and extremely low P-values (close to zero). This indicates a very strong and statistically significant relationship between these specific categorical attributes and the mushroom's `class` (poisonous or edible).

*   **Strong Predictors**: Several other features, including `cap-surface_k`, `gill-color_w`, `gill-color_n`, and `cap-shape_b`, also show high Chi2 scores and very low P-values, suggesting they are highly influential in predicting the target variable.

*   **Lowest Importance**: Features like `cap-shape_s`, `stem-color_l`, `does-bruise-or-bleed_f`, `gill-attachment_x`, and `does-bruise-or-bleed_t` show very low Chi2 scores and high P-values (closer to 1), indicating a weak or no statistically significant relationship with the target variable.

**Conclusion:**
The Chi-Square test successfully identified the most influential categorical features. These highly correlated features are strong candidates for inclusion in predictive models, while features with low scores and high P-values might be considered less important or even removed to reduce model complexity without significant loss of predictive power.

# Modelling

# Decision tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


# --- Data Preparation and Preprocessing ---
# Separate features (X) from the target (y) from the cleaned df
X_raw = df.drop('class', axis=1)
y_raw = df['class']


# Encode target variable y numerically (assuming label_encoder is already fitted)
y_encoded = label_encoder.fit_transform(y_raw)


# Apply the preprocessor to X (scaling numerical, one-hot encoding categorical)
# Ensure output is dense (preprocessor might output sparse)
X_processed_dense = preprocessor.fit_transform(X_raw)
if hasattr(X_processed_dense, 'toarray'):
    X_processed_dense = X_processed_dense.toarray()


# 1. Split the data into training and testing sets (using all features)
X_train, X_test, y_train, y_test = train_test_split(
    X_processed_dense, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")


# 2. Train Decision Tree Model with hyperparameters directly
model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    splitter='best',
    random_state=42
)


print("\nTraining Decision Tree model...")
model.fit(X_train, y_train)


print(f"\n{'='*70}")
print("HYPERPARAMETERS USED")
print(f"{'='*70}")
print(f"  - criterion: gini")
print(f"  - max_depth: 10")
print(f"  - min_samples_split: 5")
print(f"  - min_samples_leaf: 2")
print(f"  - max_features: sqrt")
print(f"  - splitter: best")


# 3. Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]


# 4. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
f1 = f1_score(y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
class_report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)


print(f"\n{'='*70}")
print("DECISION TREE MODEL EVALUATION")
print(f"{'='*70}")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print(f"\nConfusion Matrix:\n{conf_matrix}")
print(f"\nClassification Report:\n{class_report}")


# 5. Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree Confusion Matrix')
plt.tight_layout()
plt.show()


# Naive bayes model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Split the data (all preprocessed features) into training and testing sets
# Using X_processed (all preprocessed features) and y_encoded (numerically encoded target)
X_train_nb_full, X_test_nb_full, y_train_nb_full, y_test_nb_full = train_test_split(
    X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Convert sparse matrices to dense arrays for GaussianNB
# X_processed is already sparse due to OneHotEncoder.toarray()
# We should convert the split training and testing sets
if hasattr(X_train_nb_full, 'toarray'):
    X_train_nb_full = X_train_nb_full.toarray()
if hasattr(X_test_nb_full, 'toarray'):
    X_test_nb_full = X_test_nb_full.toarray()

print(f"Shape of X_train_nb_full: {X_train_nb_full.shape}")
print(f"Shape of X_test_nb_full: {X_test_nb_full.shape}")
print(f"Shape of y_train_nb_full: {y_train_nb_full.shape}")
print(f"Shape of y_test_nb_full: {y_test_nb_full.shape}")

# 2. Train Gaussian Naive Bayes Model with all features
# Instantiate a Gaussian Naive Bayes classifier
nb_model_full = GaussianNB()

# Train the model using the training data
nb_model_full.fit(X_train_nb_full, y_train_nb_full)

print("\nGaussian Naive Bayes model trained successfully with all preprocessed features.")

# 3. Evaluate Naive Bayes Model
# Make predictions on the test set
y_pred_nb_full = nb_model_full.predict(X_test_nb_full)
y_pred_proba_nb_full = nb_model_full.predict_proba(X_test_nb_full)[:, 1] # Probability of the positive class (class 'p' or 1)

# Evaluate the model
accuracy_nb_full = accuracy_score(y_test_nb_full, y_pred_nb_full)
precision_nb_full = precision_score(y_test_nb_full, y_pred_nb_full, pos_label=1)
recall_nb_full = recall_score(y_test_nb_full, y_pred_nb_full, pos_label=1)
f1_nb_full = f1_score(y_test_nb_full, y_pred_nb_full, pos_label=1)
roc_auc_nb_full = roc_auc_score(y_test_nb_full, y_pred_proba_nb_full)
conf_matrix_nb_full = confusion_matrix(y_test_nb_full, y_pred_nb_full, labels=[0, 1]) # 0 for 'e', 1 for 'p'
class_report_nb_full = classification_report(y_test_nb_full, y_pred_nb_full, target_names=label_encoder.classes_)

print(f"\nGaussian Naive Bayes Model Evaluation (using ALL preprocessed features):")
print(f"Accuracy: {accuracy_nb_full:.4f}")
print(f"Precision: {precision_nb_full:.4f}")
print(f"Recall: {recall_nb_full:.4f}")
print(f"F1-Score: {f1_nb_full:.4f}")
print(f"ROC-AUC: {roc_auc_nb_full:.4f}")
print(f"\nConfusion Matrix:\n{conf_matrix_nb_full}")
print(f"\nClassification Report:\n{class_report_nb_full}")

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_nb_full, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Gaussian Naive Bayes Confusion Matrix (All Preprocessed Features)')
plt.show()

## Gaussian Naive Bayes Model Performance Summary (All Preprocessed Features)

The Gaussian Naive Bayes model was trained and evaluated using all preprocessed features. Its performance is as follows:

### 1. Overall Performance
- **Accuracy: 0.7183**
- **Precision (for poisonous 'p'): 0.8189**
- **Recall (for poisonous 'p'): 0.6206**
- **F1-Score (for poisonous 'p'): 0.7061**
- **ROC-AUC: 0.8029**

### 2. Key Insights from the Confusion Matrix
```
[[3849  758]
 [2095 3427]]
```
- **True Negatives (TN): 3849** (Edible correctly classified as edible)
- **False Positives (FP): 758** (Edible incorrectly classified as poisonous)
- **False Negatives (FN): 2095** (Poisonous incorrectly classified as edible)
- **True Positives (TP): 3427** (Poisonous correctly classified as poisonous)

### 3. Suitability for the Business Case
**The Gaussian Naive Bayes model is not suitable for the FDA's safety-critical mission.** The most critical metric for the FDA is minimizing **False Negatives**, as these represent poisonous mushrooms that are incorrectly labeled as safe, posing a severe public health risk. With **2095 False Negatives**, this model performs significantly worse than both the Logistic Regression and Decision Tree models in this crucial aspect. Its low recall for the poisonous class (0.6206) makes it a high-risk option for deployment.

## Decision Tree Model Performance Summary (K-Best Features, Tuned Hyperparameters)

The Decision Tree model was trained on the K-best selected features with specific hyperparameters (`max_depth=10`, `min_samples_split=5`, `min_samples_leaf=2`, `max_features='sqrt'`). Here's its evaluation:

### 1. Overall Performance
- **Accuracy: 0.7855**
- **Precision (for poisonous 'p'): 0.9273**
- **Recall (for poisonous 'p'): 0.6581**
- **F1-Score (for poisonous 'p'): 0.7698**
- **ROC-AUC: 0.8733**

### 2. Key Insights from the Confusion Matrix
```
[[4322  285]
 [1888 3634]]
```
- **True Negatives (TN): 4322** (Edible correctly classified as edible)
- **False Positives (FP): 285** (Edible incorrectly classified as poisonous)
- **False Negatives (FN): 1888** (Poisonous incorrectly classified as edible)
- **True Positives (TP): 3634** (Poisonous correctly classified as poisonous)

### 3. Suitability for the Business Case
**This Decision Tree model, with these specific hyperparameters, is not suitable for the FDA's safety mission.** Despite a decent overall accuracy, the **1888 False Negatives** are still far too high for a safety-critical application. This performance is a step down from a Decision Tree with default parameters (which had only 45 FNs) and is worse even than the initial Logistic Regression model. It indicates that these specific hyperparameter choices led to a model that is less effective at identifying poisonous mushrooms, leading to unacceptable public health risks.

# SVC

In [ ]:
from sklearn.model_selection import train_test_split

# Ensure X_processed is dense for SVC
if hasattr(X_processed, 'toarray'):
    X_processed_dense = X_processed.toarray()
else:
    X_processed_dense = X_processed

X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(
    X_processed_dense,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"Shape of X_train_svc: {X_train_svc.shape}")
print(f"Shape of X_test_svc: {X_test_svc.shape}")
print(f"Shape of y_train_svc: {y_train_svc.shape}")
print(f"Shape of y_test_svc: {y_test_svc.shape}")


In [ ]:
from sklearn.svm import SVC

svc_model = SVC(
    C=0.1,
    gamma=0.01,
    kernel='rbf',
    random_state=42,
    probability=True
)

print("\nTraining SVC model (C=0.1, gamma=0.01, kernel='rbf')...")
svc_model.fit(X_train_svc, y_train_svc)
print("\nSVC model trained successfully.")


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# Predictions
y_pred_svc = svc_model.predict(X_test_svc)
y_pred_proba_svc = svc_model.predict_proba(X_test_svc)[:, 1]

# Metrics
accuracy_svc = accuracy_score(y_test_svc, y_pred_svc)
precision_svc = precision_score(y_test_svc, y_pred_svc, pos_label=1)
recall_svc = recall_score(y_test_svc, y_pred_svc, pos_label=1)
f1_svc = f1_score(y_test_svc, y_pred_svc, pos_label=1)
roc_auc_svc = roc_auc_score(y_test_svc, y_pred_proba_svc)
conf_matrix_svc = confusion_matrix(y_test_svc, y_pred_svc, labels=[0, 1])
class_report_svc = classification_report(
    y_test_svc, y_pred_svc, target_names=label_encoder.classes_
)

print(f"\nSVC Model Evaluation (C=0.1, gamma=0.01, kernel='rbf'):")
print(f"Accuracy:  {accuracy_svc:.4f}")
print(f"Precision: {precision_svc:.4f}")
print(f"Recall:    {recall_svc:.4f}")
print(f"F1-Score:  {f1_svc:.4f}")
print(f"ROC-AUC:   {roc_auc_svc:.4f}")
print(f"\nConfusion Matrix:\n{conf_matrix_svc}")
print(f"False Negatives (Critical for FDA): {conf_matrix_svc[1, 0]}")
print(f"\nClassification Report:\n{class_report_svc}")

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_svc,
    annot=True,
    fmt='d',
    cmap='Blues',
    cbar=True,
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('SVC Confusion Matrix (C=0.1, gamma=0.01)')
plt.show()


Here is the corrected summary for that specific SVC model:

SVC Model Evaluation (C=0.1, gamma=0.01, kernel='rbf'):

Accuracy: 0.8537
Precision (for poisonous 'p'): 0.8587
Recall (for poisonous 'p'): 0.8758
F1-Score: 0.8671
ROC-AUC: 0.9063
Confusion Matrix:

[[3811  796]
 [ 686 4836]]
False Negatives (FN): 686 - This means 686 poisonous mushrooms were incorrectly classified as edible.
Suitability for Business Case: While the model shows good overall metrics, these 686 False Negatives are still a concern for the FDA's safety mission. This number is lower than the initial Logistic Regression model (945 FNs) and significantly better than both Naive Bayes models (2095 FNs with all features, 4371 FNs with Lasso features). However, it is still substantially higher than the best-performing Decision Tree (45 FNs) and KNN (1 FN) models.

# naive bayes with k = 15

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Ensure dense features
X_dense = X_processed.toarray() if hasattr(X_processed, "toarray") else X_processed

# 2. Select top 15 features with KBest
selector = SelectKBest(score_func=f_classif, k=15)
X_sel = selector.fit_transform(X_dense, y_encoded)

selected_idx = selector.get_support(indices=True)
selected_features = [all_feature_names[i] for i in selected_idx]
print("Selected features:", selected_features)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 4. Train Gaussian Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)

# 5. Predictions and probabilities
y_pred = nb.predict(X_test)
y_proba = nb.predict_proba(X_test)[:, 1]

# 6. Metrics
accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall    = recall_score(y_test, y_pred, pos_label=1)
f1        = f1_score(y_test, y_pred, pos_label=1)
roc_auc   = roc_auc_score(y_test, y_proba)

print("\n=== GaussianNB with KBest (k=15) ===")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC-AUC  : {roc_auc:.4f}")

# 7. Confusion matrix + classification report
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
fn = cm[1, 0]
print("\nConfusion matrix:\n", cm)
print(f"False negatives (poisonous → edible): {fn}")

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# 8. Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(
    cm, annot=True, fmt="d", cmap="Blues",
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("GaussianNB Confusion Matrix (KBest k=15)")
plt.tight_layout()
plt.show()


## Naive Bayes Model Performance Summary (K-Best k=15 Features)

The Gaussian Naive Bayes model was trained using the top 15 features selected by `SelectKBest` with `f_classif`. Its performance on the test set is summarized below:

### 1. Overall Performance
- **Accuracy: 0.5710**
- **Precision (for poisonous 'p'): 0.9318**
- **Recall (for poisonous 'p'): 0.2300**
- **F1-Score (for poisonous 'p'): 0.3689**
- **ROC-AUC: 0.7696**

### 2. Key Insights from the Confusion Matrix
```
[[4514   93]
 [4252 1270]]
```
-   **True Negatives (TN): 4514** (Edible correctly classified as edible)
-   **False Positives (FP): 93** (Edible incorrectly classified as poisonous)
-   **False Negatives (FN): 4252** (Poisonous incorrectly classified as edible)
-   **True Positives (TP): 1270** (Poisonous correctly classified as poisonous)

### 3. Suitability for the Business Case
**The Gaussian Naive Bayes model with K-best (k=15) feature selection is highly unsuitable for the FDA's safety-critical mission.** The most critical metric for this business case is the minimization of **False Negatives**, which represent poisonous mushrooms incorrectly classified as edible. This model produced an alarmingly high **4252 False Negatives**, meaning approximately 77% of actual poisonous mushrooms in the test set were missed (Recall for poisonous 'p' is only 0.2300).

This performance is significantly worse than other models considered (Logistic Regression, Decision Tree, SVC, KNN) and poses an extreme public health risk. While its precision for the poisonous class is relatively high, its extremely poor recall makes it a dangerous option for deployment in a scenario where missing poisonous items has severe consequences.

# Naive bayes with lasso alpha(0.05)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Set the Lasso regularization strength
user_alpha = 0.05

# 2. Ensure X_processed is dense (it's already handled as a dense array or converts to dense in previous steps)
# For safety, explicitly convert if it's not already dense
X_dense = X_processed.toarray() if hasattr(X_processed, 'toarray') else X_processed

# 3. Scale the X_dense data using StandardScaler(with_mean=False)
scaler = StandardScaler(with_mean=False)
X_lasso_scaled = scaler.fit_transform(X_dense)

print(f"Shape of X_lasso_scaled: {X_lasso_scaled.shape}")

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# Reuse user_alpha and X_lasso_scaled from previous step

# 4. Initialize a Lasso model
lasso_model = Lasso(alpha=user_alpha, random_state=42, max_iter=10000)

# 5. Fit the Lasso model
lasso_model.fit(X_lasso_scaled, y_encoded)

# 6. Identify selected features (non-zero coefficients)
selected_lasso_indices = np.where(lasso_model.coef_ != 0)[0]

# 7. Create a new feature set with only these features from X_lasso_scaled
X_selected_lasso_features = X_lasso_scaled[:, selected_lasso_indices]

# Get feature names after preprocessing (using all_feature_names from previous steps)
selected_lasso_feature_names = [all_feature_names[i] for i in selected_lasso_indices]
print("Selected features by Lasso:", selected_lasso_feature_names)

# 8. Split the selected features and target into training and testing sets
X_train_lasso_nb, X_test_lasso_nb, y_train_lasso_nb, y_test_lasso_nb = train_test_split(
    X_selected_lasso_features, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nShape of X_train_lasso_nb: {X_train_lasso_nb.shape}")
print(f"Shape of X_test_lasso_nb: {X_test_lasso_nb.shape}")
print(f"Shape of y_train_lasso_nb: {y_train_lasso_nb.shape}")
print(f"Shape of y_test_lasso_nb: {y_test_lasso_nb.shape}")

# 9. Initialize a GaussianNB model
nb_model_lasso = GaussianNB()

# 10. Train the GaussianNB model
nb_model_lasso.fit(X_train_lasso_nb, y_train_lasso_nb)

print("\nGaussian Naive Bayes model trained successfully with Lasso-selected features.")

# 11. Make predictions and predict probabilities
y_pred_lasso_nb = nb_model_lasso.predict(X_test_lasso_nb)
y_pred_proba_lasso_nb = nb_model_lasso.predict_proba(X_test_lasso_nb)[:, 1] # Probability of the positive class (1 for 'p')

# 12. Calculate and print evaluation metrics
accuracy_lasso_nb = accuracy_score(y_test_lasso_nb, y_pred_lasso_nb)
precision_lasso_nb = precision_score(y_test_lasso_nb, y_pred_lasso_nb, pos_label=1)
recall_lasso_nb = recall_score(y_test_lasso_nb, y_pred_lasso_nb, pos_label=1)
f1_lasso_nb = f1_score(y_test_lasso_nb, y_pred_lasso_nb, pos_label=1)
roc_auc_lasso_nb = roc_auc_score(y_test_lasso_nb, y_pred_proba_lasso_nb)

print(f"\n=== Gaussian Naive Bayes Model Evaluation (Lasso alpha={user_alpha}) ===")
print(f"Accuracy : {accuracy_lasso_nb:.4f}")
print(f"Precision: {precision_lasso_nb:.4f}")
print(f"Recall   : {recall_lasso_nb:.4f}")
print(f"F1-score : {f1_lasso_nb:.4f}")
print(f"ROC-AUC  : {roc_auc_lasso_nb:.4f}")

# 13. Generate and print confusion matrix, explicitly noting false negatives
conf_matrix_lasso_nb = confusion_matrix(y_test_lasso_nb, y_pred_lasso_nb, labels=[0, 1])
false_negatives_lasso_nb = conf_matrix_lasso_nb[1, 0]
print("\nConfusion Matrix:\n", conf_matrix_lasso_nb)
print(f"False Negatives (poisonous \u2192 edible) - CRITICAL: {false_negatives_lasso_nb}")

# 14. Print classification report
class_report_lasso_nb = classification_report(y_test_lasso_nb, y_pred_lasso_nb, target_names=label_encoder.classes_)
print("\nClassification Report:\n", class_report_lasso_nb)

# 15. Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(
    conf_matrix_lasso_nb, annot=True, fmt="d", cmap="Blues",
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title(f"GaussianNB Confusion Matrix (Lasso alpha={user_alpha})")
plt.tight_layout()
plt.show()

## Naive Bayes with Lasso Feature Selection (alpha=0.05) Model Performance Summary

The Gaussian Naive Bayes model was trained after performing Lasso feature selection with an `alpha` of 0.05. The model's performance on the test set is summarized below:

### 1. Overall Performance
- **Accuracy: 0.5545**
- **Precision (for poisonous 'p'): 0.8909**
- **Recall (for poisonous 'p'): 0.2084**
- **F1-Score (for poisonous 'p'): 0.3378**
- **ROC-AUC: 0.7918**

### 2. Key Insights from the Confusion Matrix
```
[[4466  141]
 [4371 1151]]
```
-   **True Negatives (TN): 4466** (Edible correctly classified as edible)
-   **False Positives (FP): 141** (Edible incorrectly classified as poisonous)
-   **False Negatives (FN): 4371** (Poisonous incorrectly classified as edible)
-   **True Positives (TP): 1151** (Poisonous correctly classified as poisonous)

### 3. Suitability for the Business Case
**The Gaussian Naive Bayes model, even with Lasso feature selection at alpha=0.05, is highly unsuitable for the FDA's safety-critical mission.** The most critical metric for this business case is the minimization of **False Negatives**, which represent poisonous mushrooms incorrectly classified as edible. This model produced an alarmingly high **4371 False Negatives**, meaning nearly 80% of actual poisonous mushrooms in the test set were missed (Recall for poisonous 'p' is only 0.2084). This performance is significantly worse than other models considered and poses an extreme public health risk. The aggressive feature selection by Lasso at this `alpha` level, combined with the Naive Bayes assumption, severely limited the model's ability to correctly identify poisonous instances. This model would be dangerous to deploy in a real-world scenario.

# DTC with k-best = 15

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. Data Preparation and Preprocessing (using existing preprocessor) ---
# Separate features (X) from the target (y) from the cleaned df
X_kbest = df.drop('class', axis=1)
y_kbest = df['class']

# Encode target variable y numerically
y_encoded = label_encoder.fit_transform(y_kbest)

# Apply the preprocessor to X (scaling numerical, one-hot encoding categorical)
# Ensure output is dense for SelectKBest and Decision Tree
X_processed_dense = preprocessor.fit_transform(X_kbest)

print(f"Shape of X_processed_dense: {X_processed_dense.shape}")
print(f"Shape of y_encoded: {y_encoded.shape}")

# --- 2. K-Best Feature Selection (k=15) ---
k = 15 # Number of top features to select
selector = SelectKBest(score_func=f_classif, k=k)

# Fit and transform the processed features to get selected features
X_selected = selector.fit_transform(X_processed_dense, y_encoded)

# Get the names of the selected features for interpretability
# Need to reconstruct all feature names after preprocessing first
num_feature_names = preprocessor.named_transformers_['num'].get_feature_names_out(numerical_features)
cat_transformer = preprocessor.named_transformers_['cat']
one_hot_feature_names = cat_transformer.get_feature_names_out(categorical_features)
all_feature_names_full = list(num_feature_names) + list(one_hot_feature_names)

selected_indices_kbest = selector.get_support(indices=True)
selected_feature_names_kbest = [all_feature_names_full[i] for i in selected_indices_kbest]

print(f"\nSelected {k} features using K-Best (f_classif): {selected_feature_names_kbest}")
print(f"Shape of X_selected after K-Best: {X_selected.shape}")

# --- 3. Data Splitting ---
X_train_dt_k15, X_test_dt_k15, y_train_dt_k15, y_test_dt_k15 = train_test_split(
    X_selected, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nShape of X_train_dt_k15: {X_train_dt_k15.shape}")
print(f"Shape of X_test_dt_k15: {X_test_dt_k15.shape}")

# --- 4. Train Decision Tree Classifier ---
dt_model_k15 = DecisionTreeClassifier(random_state=42) # Using default hyperparameters for initial assessment
dt_model_k15.fit(X_train_dt_k15, y_train_dt_k15)

print("\nDecision Tree model trained successfully with K-Best (k=15) features.")

# --- 5. Evaluate Model ---
y_pred_dt_k15 = dt_model_k15.predict(X_test_dt_k15)
y_pred_proba_dt_k15 = dt_model_k15.predict_proba(X_test_dt_k15)[:, 1] # Probability of the positive class (1 for 'p')

accuracy_dt_k15 = accuracy_score(y_test_dt_k15, y_pred_dt_k15)
precision_dt_k15 = precision_score(y_test_dt_k15, y_pred_dt_k15, pos_label=1)
recall_dt_k15 = recall_score(y_test_dt_k15, y_pred_dt_k15, pos_label=1)
f1_dt_k15 = f1_score(y_test_dt_k15, y_pred_dt_k15, pos_label=1)
roc_auc_dt_k15 = roc_auc_score(y_test_dt_k15, y_pred_proba_dt_k15)

print(f"\n{'='*70}")
print("DECISION TREE MODEL EVALUATION (K-Best k=15 Features)")
print(f"{'='*70}")
print(f"Accuracy:  {accuracy_dt_k15:.4f}")
print(f"Precision: {precision_dt_k15:.4f}")
print(f"Recall:    {recall_dt_k15:.4f}")
print(f"F1-Score:  {f1_dt_k15:.4f}")
print(f"ROC-AUC:   {roc_auc_dt_k15:.4f}")

# Confusion Matrix and False Negatives
conf_matrix_dt_k15 = confusion_matrix(y_test_dt_k15, y_pred_dt_k15, labels=[0, 1]) # 0 for 'e', 1 for 'p'
false_negatives_dt_k15 = conf_matrix_dt_k15[1, 0]

print(f"\nConfusion Matrix:\n{conf_matrix_dt_k15}")
print(f"False Negatives (actual poisonous classified as edible) - CRITICAL: {false_negatives_dt_k15}")

# Classification Report
class_report_dt_k15 = classification_report(y_test_dt_k15, y_pred_dt_k15, target_names=label_encoder.classes_)
print(f"\nClassification Report:\n{class_report_dt_k15}")

# --- 6. Plot Confusion Matrix ---
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_dt_k15,
    annot=True,
    fmt='d',
    cmap='Blues',
    cbar=True,
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree Confusion Matrix (K-Best k=15 Features)')
plt.tight_layout()
plt.show()

Decision Tree Model Performance Summary (K-Best k=15 Features)
The Decision Tree model was trained using the top 15 features selected by SelectKBest with f_classif. Its performance on the test set is summarized below:

1. Overall Performance
Accuracy: 0.9047
Precision (for poisonous 'p'): 0.9130
Recall (for poisonous 'p'): 0.9122
F1-Score (for poisonous 'p'): 0.9126
ROC-AUC: 0.9041
2. Key Insights from the Confusion Matrix
[[4127  480]
 [ 485 5037]]
True Negatives (TN): 4127 (Edible correctly classified as edible)
False Positives (FP): 480 (Edible incorrectly classified as poisonous)
False Negatives (FN): 485 (Poisonous incorrectly classified as edible)
True Positives (TP): 5037 (Poisonous correctly classified as poisonous)
3. Suitability for the Business Case
This Decision Tree model, using K-best (k=15) features and default hyperparameters, performs better than the Logistic Regression model (945 FNs) and the Naive Bayes model (2095 FNs) in terms of reducing False Negatives. However, it is not as strong as the previous Decision Tree model (45 FNs) or the SVC/KNN models (1 FN each). The 485 False Negatives are still a concern for the FDA, as they represent a substantial number of poisonous mushrooms being missed. While an improvement over some models, this performance is not ideal for a safety-critical application where minimizing false negatives is paramount.

Conclusion: This Decision Tree model with K-best (k=15) features offers a reasonable balance, but its 485 False Negatives indicate that it still carries a considerable risk for the FDA's toxicity detection business case. Further optimization or considering the superior-performing SVC and KNN models would be advisable for this high-stakes application.

# Decision tree classifier with lasso (alpha 0.05)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Define the list of selected features from Lasso (alpha=0.05)
lasso_alpha_0_05_features = [
    'cap-diameter', 'stem-width', 'cap-surface_k', 'cap-color_e', 'cap-color_n',
    'cap-color_r', 'gill-attachment_p', 'gill-spacing_c', 'gill-spacing_d',
    'gill-color_w', 'stem-color_w', 'ring-type_z', 'season_w'
]

# 2. Get the indices of these selected features from all_feature_names
selected_lasso_indices_for_dt = [all_feature_names.index(f) for f in lasso_alpha_0_05_features if f in all_feature_names]

# 3. Ensure X_processed is dense and create X_selected_by_lasso_for_dt
# X_processed is already a numpy array, but we ensure it's dense if it was sparse
X_processed_dense = X_processed.toarray() if hasattr(X_processed, 'toarray') else X_processed
X_selected_by_lasso_for_dt = X_processed_dense[:, selected_lasso_indices_for_dt]

print(f"Selected features for Decision Tree with Lasso (alpha=0.05): {lasso_alpha_0_05_features}")
print(f"Shape of X_selected_by_lasso_for_dt: {X_selected_by_lasso_for_dt.shape}")

# 4. Split the data into training and testing sets
X_train_dt_lasso, X_test_dt_lasso, y_train_dt_lasso, y_test_dt_lasso = train_test_split(
    X_selected_by_lasso_for_dt, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nShape of X_train_dt_lasso: {X_train_dt_lasso.shape}")
print(f"Shape of X_test_dt_lasso: {X_test_dt_lasso.shape}")
print(f"Shape of y_train_dt_lasso: {y_train_dt_lasso.shape}")
print(f"Shape of y_test_dt_lasso: {y_test_dt_lasso.shape}")

# 5. Instantiate and train a Decision Tree Classifier
dt_model_lasso = DecisionTreeClassifier(random_state=42)
dt_model_lasso.fit(X_train_dt_lasso, y_train_dt_lasso)

print("\nDecision Tree model trained successfully with Lasso-selected features (alpha=0.05).")

# 6. Make predictions
y_pred_dt_lasso = dt_model_lasso.predict(X_test_dt_lasso)
y_pred_proba_dt_lasso = dt_model_lasso.predict_proba(X_test_dt_lasso)[:, 1]

# 7. Calculate and print evaluation metrics
accuracy_dt_lasso = accuracy_score(y_test_dt_lasso, y_pred_dt_lasso)
precision_dt_lasso = precision_score(y_test_dt_lasso, y_pred_dt_lasso, pos_label=1)
recall_dt_lasso = recall_score(y_test_dt_lasso, y_pred_dt_lasso, pos_label=1)
f1_dt_lasso = f1_score(y_test_dt_lasso, y_pred_dt_lasso, pos_label=1)
roc_auc_dt_lasso = roc_auc_score(y_test_dt_lasso, y_pred_proba_dt_lasso)

print(f"\n{'='*70}")
print("DECISION TREE MODEL EVALUATION (Lasso alpha=0.05 Features)")
print(f"{'='*70}")
print(f"Accuracy:  {accuracy_dt_lasso:.4f}")
print(f"Precision: {precision_dt_lasso:.4f}")
print(f"Recall:    {recall_dt_lasso:.4f}")
print(f"F1-Score:  {f1_dt_lasso:.4f}")
print(f"ROC-AUC:   {roc_auc_dt_lasso:.4f}")

# 8. Compute and print confusion matrix and false negatives
conf_matrix_dt_lasso = confusion_matrix(y_test_dt_lasso, y_pred_dt_lasso, labels=[0, 1])
false_negatives_dt_lasso = conf_matrix_dt_lasso[1, 0]

print(f"\nConfusion Matrix:\n{conf_matrix_dt_lasso}")
print(f"False Negatives (actual poisonous classified as edible) - CRITICAL: {false_negatives_dt_lasso}")

# 9. Generate and print classification report
class_report_dt_lasso = classification_report(y_test_dt_lasso, y_pred_dt_lasso, target_names=label_encoder.classes_)
print(f"\nClassification Report:\n{class_report_dt_lasso}")

# 10. Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_dt_lasso,
    annot=True,
    fmt='d',
    cmap='Blues',
    cbar=True,
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree Confusion Matrix (Lasso alpha=0.05 Features)')
plt.tight_layout()
plt.show()

## Decision Tree Model Performance Summary (Lasso alpha=0.05 Features)

The Decision Tree model was trained using the 13 features selected by Lasso regularization with an `alpha` value of 0.05. Its performance on the test set is summarized below:

#### 1. Overall Performance
-   **Accuracy: 0.8772**
-   **Precision (for poisonous 'p'): 0.8875**
-   **Recall (for poisonous 'p'): 0.8872**
-   **F1-Score (for poisonous 'p'): 0.8873**
-   **ROC-AUC: 0.8764**

#### 2. Key Insights from the Confusion Matrix
```
[[3986  621]
 [ 623 4899]]
```
-   **True Negatives (TN): 3986** (Edible mushrooms correctly classified as edible)
-   **False Positives (FP): 621** (Edible mushrooms incorrectly classified as poisonous)
-   **False Negatives (FN): 623** (Poisonous mushrooms incorrectly classified as edible)
-   **True Positives (TP): 4899** (Poisonous mushrooms correctly classified as poisonous)

#### 3. Suitability for the Business Case
This Decision Tree model, utilizing features selected by Lasso (alpha=0.05), shows decent overall performance metrics. An accuracy of 87.72% and an F1-score of 0.8873 for the poisonous class are generally good.

However, for the FDA's critical business case of toxicity detection, the **number of False Negatives (FN)** is the most crucial metric. In this model, there are **623 instances where a poisonous mushroom was incorrectly classified as edible.** This is a significant concern as these missed poisonous mushrooms could lead to severe public health consequences.

While 623 False Negatives are fewer than the initial Logistic Regression model (945 FNs) and the Naive Bayes models (2095 FNs with all features, 4371 FNs with Lasso features), it is still substantially higher than the best-performing models (like KNN which had only 10 FNs or the SVC which had 686 FNs, or even the Decision Tree with default parameters on K-best k=15 features which had 485 FNs).

**Conclusion:** Despite the benefits of feature selection, this Decision Tree model with Lasso-selected features (alpha=0.05) **is not yet suitable for direct deployment in a safety-critical environment like FDA toxicity detection.** The number of False Negatives remains too high, posing an unacceptable risk to public safety. For this application, further efforts should focus on models that can achieve significantly lower False Negative counts, even if it means sacrificing some precision or overall accuracy for the 'edible' class. The current performance indicates that the chosen features, while important, or the model's structure, are not sufficiently robust to minimize this critical error type.

# Model choices

KNN was overfitting no matter how many hyperparameters or modifications made for the model so we decided to take that off.

# Comparison table to find the best model

In [ ]:
import pandas as pd
from IPython.display import display

# Data for the comparison table
comparison_data = {
    'Model': [
        'Logistic Regression',
        'Decision Tree (K-best k=15)',
        'Decision Tree (Lasso alpha=0.05)',
        'Gaussian Naive Bayes (All Features)',
        'Gaussian Naive Bayes (K-best k=15)',
        'Gaussian Naive Bayes (Lasso alpha=0.05)',
        'SVC (C=0.1, gamma=0.01, kernel=\'rbf\')'
    ],
    'Accuracy': [
        0.8069,
        0.9047,
        0.8772,
        0.7183,
        0.5710,
        0.5545,
        0.8537
    ],
    'Precision (p)': [
        0.8189,
        0.9130,
        0.8875,
        0.8189,
        0.9318,
        0.8909,
        0.8587
    ],
    'Recall (p)': [
        0.8287,
        0.9122,
        0.8872,
        0.6206,
        0.2300,
        0.2084,
        0.8758
    ],
    'F1-Score (p)': [
        0.8238,
        0.9126,
        0.8873,
        0.7061,
        0.3689,
        0.3378,
        0.8671
    ],
    'ROC-AUC': [
        0.8851,
        0.9041,
        0.8764,
        0.8029,
        0.7696,
        0.7918,
        0.9063
    ],
    'False Negatives (FN)': [
        945,
        485,
        623,
        2095,
        4252,
        4371,
        686
    ]
}

comparison_df = pd.DataFrame(comparison_data)

print("\n--- Model Comparison Summary ---\n")
display(comparison_df)

print("\n--- Conclusion: The Best Model ---\n")
print("Given the critical requirement of minimizing False Negatives for the FDA's toxicity detection business case, the Decision Tree model (K-best k=15) now stands out as the best performing model among the remaining options. It achieved the lowest number of False Negatives (485) and strong overall performance, offering a much safer solution compared to Logistic Regression and Naive Bayes models.")

# Tuning hyperparameters

# Best model : DTC with k-best (k=15) = accuracy: 0.9047

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Features, target, preprocessing
X = df.drop('class', axis=1)
y = df['class']

y_encoded = label_encoder.fit_transform(y)
X_proc = preprocessor.fit_transform(X)

# 2. K-Best (k=15) – same as before but shorter
selector = SelectKBest(score_func=f_classif, k=15)
X_sel = selector.fit_transform(X_proc, y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 3. Decision Tree with fixed hyperparameters (FN-focused)
dt_model = DecisionTreeClassifier(
    random_state=42,
    criterion='gini',
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight={0: 1, 1: 3}   # poisonous (1) 3x more important
)

dt_model.fit(X_train, y_train)

# 4. Metrics and confusion matrix
y_pred = dt_model.predict(X_test)
y_proba = dt_model.predict_proba(X_test)[:, 1]

accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall    = recall_score(y_test, y_pred, pos_label=1)
f1        = f1_score(y_test, y_pred, pos_label=1)
roc_auc   = roc_auc_score(y_test, y_proba)

cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
false_negatives = cm[1, 0]  # actual poisonous, predicted edible

print("\nDecision Tree (fixed hyperparameters, k=15 K-Best):")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}  <-- key for FDA")
print(f"F1-score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("\nConfusion Matrix (rows: actual e,p; cols: predicted e,p):")
print(cm)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("False Negatives (poisonous as edible) – CRITICAL:", false_negatives)

# 5. Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    cbar=True,
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree Confusion Matrix (k=15, FN-focused)')
plt.tight_layout()
plt.show()


- The first tuning lowered the false negatives from 485 to 110.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Features, target, preprocessing
X = df.drop('class', axis=1)
y = df['class']

y_encoded = label_encoder.fit_transform(y)
X_proc = preprocessor.fit_transform(X)

# 2. K-Best (k=15)
selector = SelectKBest(score_func=f_classif, k=15)
X_sel = selector.fit_transform(X_proc, y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 3. Decision Tree tuned to reduce false negatives further
dt_model = DecisionTreeClassifier(
    random_state=42,
    max_depth=12,                 # allow a bit more complexity
    class_weight={0: 1, 1: 8}  # poisonous (1) 8x more important
)

dt_model.fit(X_train, y_train)

# 4. Metrics and confusion matrix
y_pred = dt_model.predict(X_test)
y_proba = dt_model.predict_proba(X_test)[:, 1]

accuracy  = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label=1)
recall    = recall_score(y_test, y_pred, pos_label=1)
f1        = f1_score(y_test, y_pred, pos_label=1)
roc_auc   = roc_auc_score(y_test, y_proba)

cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
false_negatives = cm[1, 0]  # actual poisonous, predicted edible

print("\nDecision Tree (FN-focused, stronger tuning):")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}  <-- aim to increase")
print(f"F1-score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("\nConfusion Matrix (rows: actual e,p; cols: predicted e,p):")
print(cm)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("False Negatives (poisonous as edible) – CRITICAL:", false_negatives)

# 5. Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    cbar=True,
    xticklabels=label_encoder.classes_,
    yticklabels=label_encoder.classes_
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Decision Tree Confusion Matrix (FN-focused, stronger tuning)')
plt.tight_layout()
plt.show()


After more tuning, our model's false negatives went from 110 to 62. with the total of the confusion matrix which is 10,129. Only 62 mushrooms could be misclassified which is better than our first tuning.

# ROC and AUC curve

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
import numpy as np # Import numpy for array operations

# Ensure X_processed_dense and y_encoded are available
# From previous steps, X_proc and y_encoded are already prepared
X_processed_dense = X_proc

print(f"Shape of X_processed_dense: {X_processed_dense.shape}")
print(f"Shape of y_encoded: {y_encoded.shape}")

# 1. & 2. & 3. Perform K-Best feature selection (k=15)
k = 15
selector_kbest = SelectKBest(score_func=f_classif, k=k)
X_sel = selector_kbest.fit_transform(X_processed_dense, y_encoded)

# 4. Retrieve names of selected features for X_sel
# Reconstruct all feature names from the preprocessor if not already done consistently
# numerical_features and categorical_features should be available from earlier steps
num_feature_names = preprocessor.named_transformers_['num'].get_feature_names_out(numerical_features)
cat_transformer = preprocessor.named_transformers_['cat']
one_hot_feature_names = cat_transformer.get_feature_names_out(categorical_features)
all_feature_names_full = np.concatenate([num_feature_names, one_hot_feature_names])

selected_indices = selector_kbest.get_support(indices=True)
selected_feature_names_kbest = [all_feature_names_full[i] for i in selected_indices]

print(f"\nSelected {k} K-Best features: {selected_feature_names_kbest}")
print(f"Shape of X_sel (K-Best selected features): {X_sel.shape}")

# 5. & 6. Create train-test split for the full feature set (X_processed_dense)
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_processed_dense, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 7. Create train-test split for the K-best selected feature set (X_sel)
X_train_kbest, X_test_kbest, y_train_kbest, y_test_kbest = train_test_split(
    X_sel, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("\n--- Train-Test Split Shapes ---")
print(f"X_train_full shape: {X_train_full.shape}")
print(f"X_test_full shape: {X_test_full.shape}")
print(f"y_train_full shape: {y_train_full.shape}")
print(f"y_test_full shape: {y_test_full.shape}")
print(f"X_train_kbest shape: {X_train_kbest.shape}")
print(f"X_test_kbest shape: {X_test_kbest.shape}")
print(f"y_train_kbest shape: {y_train_kbest.shape}")
print(f"y_test_kbest shape: {y_test_kbest.shape}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 1. Train Decision Tree Classifier with FN-focused hyperparameters on K-best data
dt_fn_focused_model = DecisionTreeClassifier(
    random_state=42,
    max_depth=12,
    class_weight={0: 1, 1: 8} # poisonous (1) 8x more important
)
dt_fn_focused_model.fit(X_train_kbest, y_train_kbest)

# Collect predicted probabilities for ROC curve
y_pred_proba_dt_fn_focused = dt_fn_focused_model.predict_proba(X_test_kbest)[:, 1]

print("Decision Tree (FN-focused) model trained on K-best data.")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 2. Train Decision Tree Classifier with default hyperparameters on K-best data
dt_default_model = DecisionTreeClassifier(random_state=42)
dt_default_model.fit(X_train_kbest, y_train_kbest)

# Collect predicted probabilities for ROC curve
y_pred_proba_dt_default = dt_default_model.predict_proba(X_test_kbest)[:, 1]

print("Decision Tree (default) model trained on K-best data.")

In [ ]:
from sklearn.svm import SVC

# 3. Train Support Vector Classifier (SVC) on the full data
svc_model = SVC(
    C=0.1,
    gamma=0.01,
    kernel='rbf',
    random_state=42,
    probability=True # Required to get predict_proba for ROC curve
)
svc_model.fit(X_train_full, y_train_full)

# Collect predicted probabilities for ROC curve
y_pred_proba_svc = svc_model.predict_proba(X_test_full)[:, 1]

print("SVC model trained on full data.")

In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Instantiate a Logistic Regression model
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

# 2. Train the model using the full training data
# X_train_full and y_train_full are available from previous steps
log_reg_model.fit(X_train_full, y_train_full)

# 3. Predict the probabilities for the positive class (class 1, which is 'p' after encoding)
y_pred_proba_log_reg = log_reg_model.predict_proba(X_test_full)[:, 1]

print("Logistic Regression model trained successfully and probabilities collected.")

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

# Calculate ROC curve and AUC for Decision Tree (FN-focused)
fpr_dt_fn, tpr_dt_fn, _ = roc_curve(y_test_kbest, y_pred_proba_dt_fn_focused)
roc_auc_dt_fn = auc(fpr_dt_fn, tpr_dt_fn)
plt.plot(fpr_dt_fn, tpr_dt_fn, color='darkorange', lw=2, label=f'DT (FN-focused) K-Best (AUC = {roc_auc_dt_fn:.4f})')

# Calculate ROC curve and AUC for Decision Tree (default)
fpr_dt_def, tpr_dt_def, _ = roc_curve(y_test_kbest, y_pred_proba_dt_default)
roc_auc_dt_def = auc(fpr_dt_def, tpr_dt_def)
plt.plot(fpr_dt_def, tpr_dt_def, color='green', lw=2, label=f'DT (default) K-Best (AUC = {roc_auc_dt_def:.4f})')

# Calculate ROC curve and AUC for SVC
fpr_svc, tpr_svc, _ = roc_curve(y_test_full, y_pred_proba_svc)
roc_auc_svc = auc(fpr_svc, tpr_svc)
plt.plot(fpr_svc, tpr_svc, color='blue', lw=2, label=f'SVC Full Data (AUC = {roc_auc_svc:.4f})')

# Calculate ROC curve and AUC for Logistic Regression (newly added)
fpr_log_reg, tpr_log_reg, _ = roc_curve(y_test_full, y_pred_proba_log_reg)
roc_auc_log_reg = auc(fpr_log_reg, tpr_log_reg)
plt.plot(fpr_log_reg, tpr_log_reg, color='red', lw=2, label=f'Logistic Regression Full Data (AUC = {roc_auc_log_reg:.4f})')

# Plot the random classifier line
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.50)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve Comparison')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

#Analysis

## ROC AUC Curve Analysis: Comparing the Best Models

The Receiver Operating Characteristic (ROC) curve is a fundamental tool for evaluating the performance of binary classifiers, illustrating the trade-off between the True Positive Rate (TPR) and False Positive Rate (FPR) at various threshold settings. The Area Under the Curve (AUC) provides a single scalar value that summarizes the overall diagnostic ability of the classifier, with higher values indicating better performance.

Our comparison includes four top-performing models:
1.  **DT (FN-focused) K-Best**: Decision Tree tuned for minimizing False Negatives, trained on K-best (k=15) features.
2.  **DT (default) K-Best**: Decision Tree with default hyperparameters, trained on K-best (k=15) features.
3.  **SVC Full Data**: Support Vector Classifier with `C=0.1, gamma=0.01, kernel='rbf'`, trained on the full preprocessed dataset.
4.  **Logistic Regression Full Data**: Logistic Regression with `max_iter=1000`, trained on the full preprocessed dataset.

### Key Observations from the ROC Curve Plot:

1.  **Overall Performance Ranking by AUC:**
    *   **DT (FN-focused) K-Best (AUC = 0.9607):** This model exhibits the highest AUC score among the four, indicating an outstanding ability to discriminate between edible and poisonous mushrooms. Its aggressive tuning to prioritize recall (minimize false negatives) has evidently resulted in a classifier that is exceptionally good at ranking poisonous instances above edible ones.
    *   **SVC Full Data (AUC = 0.9063):** The SVC model follows closely, also demonstrating excellent discriminative power. It's a robust performer, generally very good at distinguishing the two classes.
    *   **DT (default) K-Best (AUC = 0.9041):** The Decision Tree with default settings, using the same K-best features, shows performance very comparable to the SVC model, reinforcing the strength of the selected features.
    *   **Logistic Regression Full Data (AUC = 0.8818):** Logistic Regression, while slightly lower than the other top models, still presents a strong AUC, confirming its solid predictive capability and ability to separate the classes better than random chance.
    *   **Random Classifier (AUC = 0.50):** As a baseline, the random classifier serves to highlight the superior performance of all trained models.

2.  **Trade-off between False Positives and True Positives:**
    *   All four models' curves are significantly positioned above the diagonal (random classifier) line, which confirms their utility in making predictions.
    *   The **DT (FN-focused) K-Best** model's curve is notably pushed towards the top-left corner of the plot. This shape is highly desirable, especially for the FDA's use case, as it signifies that the model can achieve a very high True Positive Rate (i.e., successfully identify most poisonous mushrooms) while maintaining a very low False Positive Rate (i.e., avoid incorrectly flagging too many edible mushrooms as poisonous). This is a strong indicator of its effectiveness in a safety-critical context.
    *   The **SVC Full Data** and **DT (default) K-Best** curves are also excellent, showing a rapid increase in TPR for minimal increases in FPR, confirming their strong ability to separate classes.
    *   The **Logistic Regression Full Data** curve, while good, shows a slightly less steep ascent to high TPR compared to the top two Decision Tree models and SVC, suggesting it might incur a slightly higher FPR for the same TPR, or achieve a slightly lower TPR for a given FPR compared to the others.

3.  **Implications for FDA Business Case (Minimizing False Negatives):**
    *   For the FDA, the primary objective is to minimize False Negatives – the instances where a poisonous mushroom is mistakenly classified as edible. In the context of an ROC curve, this translates to maximizing the True Positive Rate (Recall) at very low False Positive Rates. The closer the curve is to the top-left corner, the better the model is at this task.
    *   The **DT (FN-focused) K-Best** model's curve visually demonstrates superior performance in this critical region. Its high AUC, coupled with its tuning strategy, suggests it can identify poisonous mushrooms with exceptional reliability, reducing the risk of unsafe products reaching the public.

### Conclusion:

The ROC AUC curve analysis strongly reinforces that the **Decision Tree model with FN-focused hyperparameters (tuned on K-best k=15 features)** is the most compelling candidate for the FDA's toxicity detection system. Its highest AUC score and the ideal shape of its ROC curve, particularly its ability to achieve high True Positive Rates at very low False Positive Rates, align perfectly with the safety-critical nature of this application. While the SVC and default Decision Tree models also show excellent discriminative power, the explicit optimization for minimizing false negatives in the FN-focused Decision Tree provides a crucial advantage in a scenario where the cost of a missed poisonous mushroom is extremely high.

#Business case interpretation

## Business Case: Mushroom Toxicity Detection for the FDA

**Target:** Classify mushrooms as poisonous ('p') or edible ('e').

**Business Case:** Help the FDA (Food and Drug Administration) ensure safety through toxicity detection for food and drug products that contain mushrooms. The primary objective is to **minimize False Negatives (FN)** – instances where a poisonous mushroom is incorrectly classified as edible – to prevent severe public health consequences.

### Model Selection and Tuning for FDA's Mission

1.  **Initial Model Choice: Decision Tree (K-best k=15)**
    *   **Reasoning:** At the initial comparison stage, before extensive tuning, the **Decision Tree model with K-best (k=15) features** was chosen as the best performer among the then-evaluated models. It exhibited an **Accuracy of 0.9047**, which was notably higher than other models like Logistic Regression (0.8069) and all Naive Bayes variations. More crucially, it also demonstrated a significantly lower initial False Negative count of **485** compared to Logistic Regression's 945 FNs, and all Naive Bayes models which had FNs in the thousands.

2.  **Hyperparameter Tuning for False Negative Reduction**
    *   **Goal:** Despite its initial strong performance, 485 False Negatives were still deemed too high for a safety-critical application. The paramount goal for the FDA is to minimize the risk of poisonous products reaching consumers, even if it means tolerating a slightly higher rate of False Positives (edible mushrooms incorrectly flagged as poisonous).
    *   **Tuning Strategy:** The Decision Tree model (K-best k=15) was subjected to targeted hyperparameter tuning. The focus was on adjusting parameters to make the model more sensitive to the positive class ('poisonous') and penalize false negatives more heavily.
    *   **Hyperparameters Used:**
        *   `random_state=42` (for reproducibility)
        *   `max_depth=12` (allowing for slightly more complexity)
        *   `class_weight={0: 1, 1: 8}`: This was a critical adjustment. It assigned a weight of 8 to the poisonous class (1) and 1 to the edible class (0). This tells the model that misclassifying a poisonous mushroom (False Negative) is 8 times worse than misclassifying an edible one (False Positive), thus strongly incentivizing the model to correctly identify poisonous instances.

3.  **Impact of Tuning: Dramatic Reduction in False Negatives**
    *   **Result:** Through this focused tuning, the number of False Negatives was dramatically reduced from the initial **485 FNs** down to just **62 FNs**. This represents an **87% reduction** in critical errors.
    *   **Current Performance (FN-focused Decision Tree):**
        *   Accuracy: 0.8474
        *   Precision (p): 0.7863
        *   **Recall (p): 0.9888**
        *   F1-Score (p): 0.8760
        *   ROC-AUC: 0.9565
        *   **False Negatives (FN): 62**

### How This Benefits the FDA

*   **Enhanced Public Safety:** The reduction of False Negatives from 485 to 62 is a direct and substantial benefit for the FDA. It means that the model is now significantly more reliable in identifying poisonous mushrooms, drastically decreasing the chance of contaminated products reaching the market and causing harm.
*   **Risk Mitigation:** With only 62 out of over 10,000 test samples being misclassified as edible when they are poisonous, the public health risk is minimized to an almost negligible level for this dataset. This aligns perfectly with the FDA's core mission of consumer protection.
*   **Early Detection:** The high recall for the poisonous class (0.9888) ensures that the vast majority of toxic mushrooms are flagged, allowing the FDA to take proactive measures to prevent their distribution.
*   **Operational Efficiency (Reduced False Positives):** While the tuning prioritized False Negatives, the model still maintains reasonable precision, meaning false alarms are also kept in check (though at 1484 FPs, this is a trade-off accepted for safety). This balance prevents excessive and costly recalls of safe products.

**Conclusion:** By choosing the Decision Tree model for its initial strong accuracy and then strategically tuning its hyperparameters, particularly using `class_weight` to prioritize the detection of poisonous mushrooms, we achieved a highly effective and safety-focused solution. The resulting model, with only 62 False Negatives, offers a robust tool for the FDA to significantly enhance public safety in mushroom-containing products.

The Goal: Minimizing False Negatives is Paramount: For the FDA's business case, the cost of a False Negative (a poisonous mushroom incorrectly classified as edible) is extremely high, potentially leading to severe public health consequences. Therefore, our tuning objective was explicitly to reduce False Negatives as much as possible, even if it meant sacrificing other metrics.

How class_weight Influenced This: We used the hyperparameter class_weight={0: 1, 1: 8}. This tells the Decision Tree model that misclassifying an instance of the positive class ('poisonous', encoded as 1) is 8 times more costly than misclassifying an instance of the negative class ('edible', encoded as 0). By assigning a much higher penalty to False Negatives, the model becomes much more conservative in predicting 'edible'.

The Trade-off (False Negatives vs. False Positives & Accuracy):

When the model tries extremely hard to avoid False Negatives, it often becomes more prone to generating False Positives (edible mushrooms incorrectly classified as poisonous). It essentially prefers to err on the side of caution.
Looking at the tuned model's confusion matrix, while False Negatives drastically reduced from 485 to 62, the False Positives increased significantly (from around 480 to 1484).
Accuracy is calculated as (True Positives + True Negatives) / Total. When a large number of True Negatives (correctly identified edible mushrooms) are converted into False Positives (incorrectly identified as poisonous) in an effort to reduce False Negatives, the overall accuracy tends to decrease. The model might be making more 'mistakes' overall (higher FPs) but fewer of the critical mistakes (lower FNs).
In essence, the drop in accuracy is an acceptable and strategically chosen consequence. For the FDA, preventing a single poisonous mushroom from reaching the public outweighs the operational cost of discarding or re-testing several edible mushrooms. The tuning successfully shifted the model's bias to prioritize public safety over raw predictive accuracy, which is the correct approach for this safety-critical business case.

# Scalability

## Real-World Operation and Scalability of the Decision Tree Model

For a critical application like the FDA's mushroom toxicity detection, the chosen Decision Tree model (tuned for minimizing False Negatives) needs to seamlessly integrate into existing workflows and handle varying data loads. Here's a breakdown of its real-world operation and scalability considerations:

### 1. Real-World Operation Workflow

1.  **Data Ingestion**: Raw mushroom characteristic data (e.g., cap-diameter, stem-width, gill-attachment, stem-color) from various sources (e.g., laboratory analyses, field reports, new product submissions) would be collected.
2.  **Preprocessing Pipeline**: This raw data would then pass through the exact preprocessing pipeline established in our analysis:
    *   **Missing Value Handling**: Apply the same logic used during training (e.g., mode imputation) for any features that might still have missing values in new incoming data.
    *   **Feature Engineering/Extraction**: The relevant features would be extracted. Since our model uses a reduced set of 15 K-best features, only these would be passed forward.
    *   **Scaling and Encoding**: Numerical features (like `cap-diameter`, `stem-height`, `stem-width`) would be scaled using the *fitted* `StandardScaler` from training. Categorical features would be one-hot encoded using the *fitted* `ColumnTransformer` (specifically the `OneHotEncoder`) from training to ensure consistency.
3.  **Prediction**: The preprocessed and transformed feature vector for a new mushroom sample would then be fed into the *trained* Decision Tree model.
4.  **Decision Output**: The model would output a prediction (poisonous 'p' or edible 'e') along with a probability score. Given our focus on minimizing False Negatives, a higher probability for 'poisonous' (even if below 0.5) might trigger further human inspection or automatic rejection based on a custom threshold (determined by the FDA's acceptable risk level).
5.  **Action & Reporting**: Based on the prediction and associated risk, the FDA system would trigger appropriate actions, such as approving a product, flagging it for further testing, or rejecting a batch.
6.  **Continuous Monitoring & Retraining**: The model's performance in production would be continuously monitored. If data drifts or new mushroom types emerge, the model would need to be periodically retrained with updated data to maintain its effectiveness.

### 2. Scalability for Deployment

Our chosen Decision Tree model offers several advantages concerning scalability, making it well-suited for deployment in an organization like the FDA:

*   **Computational Efficiency (Inference)**:
    *   **Low Latency**: Decision Trees are non-parametric and make predictions by traversing a tree structure. This process is extremely fast, typically requiring very low latency for individual predictions. This is critical for real-time or near real-time screening of mushroom products.
    *   **Resource-Light**: The model (once trained) has a small footprint and requires minimal computational resources (CPU, memory) for making predictions, even at high volumes.
*   **Feature Reduction**: The K-best feature selection (`k=15`) significantly reduced the input dimensionality. This means less data needs to be processed and stored for each prediction, contributing directly to better scalability and faster inference times.
*   **Robustness to Data Volume**: While training large Decision Trees can be memory-intensive, making predictions with a fixed-size, shallow Decision Tree (like ours with `max_depth=12`) scales very well with increasing numbers of prediction requests. It does not become significantly slower as the amount of data to predict on increases, as long as the data fits in memory for batch processing or can be streamed effectively.
*   **Ease of Deployment**: Decision Trees are relatively simple to serialize and deploy using standard machine learning libraries (e.g., Python's `pickle` or `joblib`) or within containerized environments (e.g., Docker, Kubernetes) that are common in enterprise-level deployments.
*   **Interpretable & Auditable**: For a regulatory body like the FDA, model interpretability is paramount. Decision Trees, with their rule-based structure, allow for clear auditing and explanation of why a particular prediction was made. This builds trust and facilitates compliance. Although not directly scalability, it eases the operational overhead of explaining model decisions.

**In summary**, the optimized Decision Tree model is not only highly effective at meeting the FDA's safety requirements by minimizing false negatives but also possesses strong characteristics for real-world operation and scalability. Its computational efficiency, reduced feature set, and interpretable nature make it a robust candidate for practical deployment.

# How to interprete the model

## Interpreting the Tuned Decision Tree Model (62 False Negatives)

Our chosen Decision Tree model, meticulously tuned to minimize False Negatives, is designed to be a critical tool for the FDA. Here's how its outputs and performance should be interpreted in a real-world context:

### 1. Model's Core Output
When presented with a new mushroom sample, the model will output:
*   **A Classification:** Either 'p' (poisonous) or 'e' (edible).
*   **A Probability Score:** The model's confidence in its classification (e.g., 0.98 for 'poisonous', 0.02 for 'edible').

### 2. The Significance of 62 False Negatives (FNs)

*   **What it Means:** Out of approximately 5522 truly poisonous mushrooms in the test set, the model incorrectly classified only 62 as 'edible'. This means it missed identifying a very small fraction (about 1.1%) of potentially dangerous mushrooms.
*   **Impact for FDA:** This is the most critical metric for the FDA. Reducing FNs from 485 to 62 signifies a monumental increase in public safety. It means the model is highly effective at preventing toxic products from being released. The risk of consumers ingesting a misclassified poisonous mushroom is significantly minimized.
*   **Actionable Insight:** While 62 FNs is excellent, it's not zero. The FDA could establish a protocol for these rare instances, such as further human inspection or highly conservative action (e.g., discarding the batch) for any mushroom where the model's confidence for 'edible' is very low, or its confidence for 'poisonous' is non-negligible, even if the final classification is 'edible'.

### 3. Understanding the Trade-off: False Positives (1484 FPs)

*   **What it Means:** The model incorrectly classified 1484 truly edible mushrooms as 'poisonous'. This means that out of approximately 4607 truly edible mushrooms in the test set, about 32.2% were flagged as dangerous.
*   **Impact for FDA:** While this number is higher than some other models, it is the deliberate cost of achieving a very low FN rate. False Positives lead to operational inefficiencies (e.g., discarding safe products, re-testing), but they **do not directly pose a public health risk**. For the FDA, this trade-off is acceptable because the safety of consumers outweighs economic costs.
*   **Actionable Insight:** The FDA could use the probability scores associated with these FP classifications. Mushrooms with a moderate probability of being 'poisonous' (predicted by the model to be FP) could be sent for secondary, more expensive testing rather than immediate disposal, thereby optimizing resources while maintaining safety.

### 4. How Feature Importance Guides Interpretation

The model's decisions are primarily driven by the features `stem-width`, `cap-diameter`, `gill-color_w`, `gill-spacing_c`, and `stem-color_w`. If a mushroom has characteristics that align with known poisonous traits for these features (e.g., a specific stem width beyond a certain threshold, or a particular gill color), the model will lean towards a 'poisonous' classification. This transparency allows domain experts at the FDA to understand *why* a decision was made.

### 5. Overall Interpretation for FDA Decision-Making

This tuned Decision Tree model provides a robust, safety-first screening tool:
*   **Conservative but Effective:** It is designed to be conservative when it comes to classifying a mushroom as 'edible'. It will err on the side of caution (classifying as 'poisonous') rather than missing a truly toxic one.
*   **Risk Mitigation:** The extremely low False Negative rate means the FDA can have high confidence that products cleared by this model are genuinely safe, significantly reducing the risk of public health incidents.
*   **Efficiency:** While not perfect, the model streamlines the initial screening process, allowing resources (human inspection, laboratory testing) to be focused on a manageable subset of potentially risky products (those flagged as 'poisonous' by the model, including the FPs, and any with uncertain probability scores).

In essence, the model acts as a highly reliable 'gatekeeper,' prioritizing public safety above all else, which is the paramount concern for the FDA.